# SERIMATS Application


|           |                |
| : ----- : | :------------: |
| Candidate | Matthieu David |
| stream    | developmental interpretability |
| subject   | [task variability](https://devinterp.com/projects/task-variability) |



This notebook is inspired from the notebook [RLCT Estimation of MNIST](https://colab.research.google.com/github/timaeus-research/devinterp/blob/main/examples/mnist.ipynb) with large change to fit my submission.


---
## RLCT Estimation of MNIST

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)]()

This Jupyter Notebook aims to reproduce the results of Lau et al. (2023) by measuring the Real Log Canonical Threshold (RLCT) for a small 2-layer ReLU model (about 1M parameters) trained on the MNIST dataset. It uses both Stochastic Gradient Nose-Hoover Thermostat (SGNHT) and Stochastic Gradient Langevin Dynamics (SGLD) as sampling methods.

---

## Table of Contents:
* packages installation
* Import for the all notebook
* Method definitions
* Experiments 1 (XP1): Impact of the number of classes
    * Toy model and Resnet18 instances
    * Settings and runs label sets $Card(\mathcal{L}abels) = 2$
    * Settings and runs label sets $Card(\mathcal{L}abels) = 3$
    * Settings and runs label sets $Card(\mathcal{L}abels) = 4$
    * Settings and runs label sets $Card(\mathcal{L}abels) = 5$
    * Settings and runs label sets $Card(\mathcal{L}abels) = 6$
    * Settings and runs label sets $Card(\mathcal{L}abels) = 10$
* Displays $λ\ vs\ \mathcal{L}abels$
* Experiments 2 (XP2): Impact of the data diversity (not perform due to time limitation in instruction)
* Experiments 3 (XP3): Impact of the number of classes (not perform due to time limitation in instruction)

---

## Package installation:

In [ ]:
%pip install devinterp matplotlib seaborn torchvision

## Import for all the notebook

In [ ]:
import copy
import os
from typing import Literal

import matplotlib.pyplot as plt
import pandas as pd
from pydantic.types import confloat, conint
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm

from torchvision import datasets, transforms
from torchvision.models import resnet18

from devinterp.optim.sgld import SGLD
from devinterp.optim.sgnht import SGNHT
from devinterp.slt import estimate_learning_coeff

PRIMARY, SECONDARY, TERTIARY = sns.color_palette("muted")[:3]
PRIMARY_LIGHT, SECONDARY_LIGHT, TERTIARY_LIGHT = sns.color_palette("pastel")[:3]
sns.set_theme("paper")

## Method definitions

#### Class of the Toy model:
The toy model corresponds to the initial model used in the notebook RLCT Estimation of MNIST (rewrite a little for lisibility). This model is made of 2 linear layers, each activations being a ReLU function:

In [ ]:

# Define the toy neural network
class OriginalToyNet(nn.Module):
    def __init__(
        self,
        hidden_layer_sizes=[1024, 1024],
        input_dim=28 * 28,
        output_dim=10,
        activation=F.relu,
        with_bias=True,
    ):
        super(OriginalToyNet, self).__init__()
        self.input_dim = input_dim
        self.layer_sizes = [input_dim] + hidden_layer_sizes + [output_dim]
        self.activation = activation
        self.with_bias = with_bias
        self.layers = nn.ModuleList()
        for i in range(len(self.layer_sizes) - 1):
            dim_in, dim_out = self.layer_sizes[i : i + 2]
            self.layers.append(nn.Linear(dim_in, dim_out, bias=self.with_bias).float())

    def forward(self, x):
        x = x.view(-1, self.input_dim)
        for layer in self.layers[:-1]:
            x = self.activation(layer(x))
        x = self.layers[-1](x)
        return x

# Define the toy neural network
class ToyNet(nn.Module):
    def __init__(
        self,
        hidden_layer_sizes: list[int]=[1024, 1024],
        input_dim: int=28 * 28,
        output_dim: int=10,
        with_bias: bool=True,
    ):
        """Toy model for task variability experiment

        Args:
            hidden_layer_sizes (list, optional): list of the size of each hidden layers. Defaults to [1024, 1024].
            input_dim (_type_, optional): Width and height of the input data. Defaults to 28*28.
            output_dim (int, optional): siez of the output layer. Corresponds to the number of classes. Defaults to 10.
            with_bias (bool, optional): allowing biais term or not for each layers. Defaults to True.
        """
        super(ToyNet, self).__init__()
        self.input_dim = input_dim
        self.with_bias = with_bias
        self.layer_sizes = [input_dim] + hidden_layer_sizes + [output_dim]

        # First basic block = Linear + ReLU activation
        self.fc_1 = nn.Linear(in_features=input_dim, out_features=1024, bias=self.with_bias)
        self.relu_1 = nn.ReLU()

        # 2nd basic block = Linear + ReLU activation
        self.fc_2 = nn.Linear(in_features=1024, out_features=1024, bias=self.with_bias)
        self.relu_2 = nn.ReLU()

        # Last Fully Connected layer:
        self.fc_3 = nn.Linear(in_features=1024, out_features=output_dim, bias=self.with_bias)



    def forward(self, x):
        """Computation performed at every call

        Args:
            x: input data (batch_size, height, width) propagated through Net

        Returns:
            output result of x through Net
        """
        x = x.view(-1, self.input_dim)

        x = self.fc_1(x)
        x = self.relu_1(x)
        x = self.fc_2(x)
        x = self.relu_2(x)
        x = self.relu_2(x)

        x = self.fc_3(x)
        return x

In [ ]:
def train_one_epoch(model, train_loader, optimizer, criterion):
    "Original train_one_epoch method from MNIST notebook of timaeus/devinterp"
    model.train()
    train_loss = 0
    for data, target in tqdm(train_loader):
        optimizer.zero_grad()
        output = model(data.to(DEVICE))
        loss = criterion(output, target.to(DEVICE))
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
    return train_loss / len(train_loader)


def evaluate(model, test_loader, criterion):
    "Original evaluate method from MNIST notebook of timaeus/devinterp"
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data.to(DEVICE))
            loss = criterion(output, target.to(DEVICE))
            test_loss += loss.item()
    return test_loss / len(test_loader)


def train_step(model: nn.Module, optimizer: optim.Optimizer, criterion:nn.modules.loss._Loss, train_set: DataLoader):
    """_summary_

    Args:
        model (nn.Module): model to train
        optimizer (optim.Optimizer): optimizer to used during training
        criterion (nn.modules.loss._Loss): Loss function to use for the training
        train_set (DataLoader): train dataloader

    Returns:
        tuple(average train loss, average train accuracy)
    """
    model.train()
    # model.to(DEVICE)

    train_loss, train_acc = 0.0, 0.0
    i = 0
    prog_bar = tqdm(train_set, desc=f"\ttrain_loss: {train_loss:0.2f} -- train_acc: {train_acc:0.2f} %")
    for x, y in prog_bar:
        # Stating variables
        x = x.to(DEVICE)
        y = y.to(DEVICE)

        optimizer.zero_grad() # Clear gradients
        output = model(x) # Forward step
        loss = criterion(output, y) # Loss step

        # Backward step
        loss.backward()
        optimizer.step()

        # Accuracy
        top_p, prediction = output.topk(1, dim=1)
        equals = (prediction == y.view(*prediction.shape))

        prog_bar.set_description(f"\ttrain_loss: {train_loss / len(train_set) :0.5f} -- train_acc: {100 * train_acc / len(train_set) :0.2f} %")
        # Saving data
        train_loss += loss.item()
        train_acc += torch.mean(equals.type(torch.FloatTensor))

    return train_loss / len(train_set), train_acc / len(train_set)


def val_step(model: nn.Module, criterion: nn.modules.loss._Loss, val_set: DataLoader):
    """ Evaluation method. Calculate the loss and accuracy on the hold out data set

    Args:
        model (nn.Module): model to evaluate
        criterion (nn.modules.loss._Loss): loss function to use
        val_set (DataLoader): evaluation dataloader

    Returns:
        _type_: _description_
    """
    val_loss, val_acc = 0.0, 0.0

    model.eval()
    # model.to(DEVICE)
    with torch.no_grad():
        for i, (x, y) in enumerate(val_set):
            # Stating variables
            images = x.to(DEVICE)
            labels = y.to(DEVICE)

            output = model(images) # Forward step

            loss = criterion(output, labels) # Loss step

            # Accuracy
            top_p, prediction = output.topk(1, dim=1)
            equals = (prediction == labels.view(*prediction.shape))

            # Saving data
            val_loss += loss.item()
            val_acc += torch.mean(equals.type(torch.FloatTensor))

    return val_loss / len(val_set), val_acc / len(val_set)

def print_step_report(epoch:int, train_loss: float, val_loss: float, train_acc: float, val_acc: float):
    """ Utility method to gather printed information"""
    print(f"Epoch {epoch+1}:\n" \
          f"Train Loss: {train_loss:.5f}" \
          f"-- Val Loss: {val_loss:.5f}" \
          f"-- Train Acc: {train_acc:0.2f}" \
          f"-- Val Acc: {val_acc:0.2f}")

def plot_lambda_vs_epochs(train_losses, test_losses, rlct_estimates_sgnht, rlct_estimates_sgld):
    """ Utility method to manage plot of losses and RLCT coefficient wrt epochs"""
    sns.set_style("whitegrid")
    fig, ax1 = plt.subplots()
    ax1.set_xlabel("Epoch")
    ax1.set_ylabel("Loss", color=PRIMARY)
    ax1.plot(train_losses, label="Train Loss", color=PRIMARY)
    ax1.plot(test_losses, label="Test Loss", color=PRIMARY_LIGHT)
    ax1.tick_params(axis="y", labelcolor=PRIMARY)
    ax1.legend(loc="lower left")

    ax2 = ax1.twinx()
    ax2.set_ylabel(r"Local Learning Coefficient, $\hat \lambda$", color=SECONDARY)
    ax2.plot(rlct_estimates_sgnht, label="SGNHT", color=SECONDARY)
    ax2.plot(rlct_estimates_sgld, label="SGLD", color=SECONDARY_LIGHT)
    ax2.tick_params(axis="y", labelcolor=SECONDARY)
    ax2.legend(loc="center right")

    fig.tight_layout()
    plt.show()

## Experiments 1 (XP1): Impact of the number of classes

This section contains all the experiments about the XP1 detailled in the application report [here](link)

First basic code cells are given the dataset MNIST and basics training with the Toy model and Resnet18 are reported. The readers can explore the core process without modifying the code for the actual experiment.

Secondly each sub experiments possess a dedicated sub section , new data loaders are generated, where the labels are rewritten to performed the experiment with the dedicated label set cardinality.

### MNIST dataset

In [ ]:
# Load MNIST data
## definition of the transformation to apply to dataset
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.244, 0.225]),
    ]
)

## Load train data
batch_size = 512
train_data = datasets.MNIST("../data", train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
# Load test data
test_data = datasets.MNIST("../data", train=False, transform=transforms.ToTensor())
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [ ]:
run_toy_model = True
run_resnet = False

### Toy model and Resnet18 instances

In [ ]:
# Depending of the run type, cuda or cpu mode is defined
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# --------------------------------------------
N_CLASSES = 10 # for the 10 class digits of MNIST
lr = 0.005
n_epochs = 20
# --------------------------------------------
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 instance    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

# Initialize model, loss, optimizer and sgld sampler
## taking an untrained resnet18
if run_resnet:
    model_resnet18 = resnet18(weights=None).to(DEVICE)

    ## changing the last fully connected layer of N neurons, each corresponding to a class.
    new_fc_act = nn.Linear(model_resnet18.fc.in_features, N_CLASSES)
    model_resnet18.fc = new_fc_act

    ## Changing the first conv2d layer to match the input image shape
    model_resnet18.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

    # Loss and optimizer definition
    criterion_resnet18 = nn.CrossEntropyLoss()
    optimizer_resnet18 = optim.SGD(model_resnet18.parameters(), lr=lr, momentum=0.9, nesterov=True)

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy Model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

if run_toy_model:
    # Initialize model, loss, optimizer and sgld sampler
    model_toy = ToyNet().to(DEVICE)
    criterion_toy = nn.CrossEntropyLoss()
    optimizer_toy = optim.SGD(model_toy.parameters(), lr=lr, momentum=0.9, nesterov=True)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy Model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

if run_toy_model:
    train_losses_toy = []
    val_losses_toy = []
    models_toy = []
    for epoch in range(n_epochs):
        train_loss, train_acc = train_step(model_toy,
                                        optimizer_toy,
                                        criterion_toy,
                                        train_loader)
        val_loss, val_acc = val_step(model_toy,
                                    criterion_toy,
                                    test_loader)
        train_losses_toy.append(train_loss)
        val_losses_toy.append(val_loss)
        models_toy += [copy.deepcopy(model_toy)]

        print_step_report(epoch=epoch,
                        train_loss=train_loss,
                        val_loss=val_loss,
                        train_acc=train_acc,
                        val_acc=val_acc)

    torch.save(models_toy, "toy_models.pth")

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 instance    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

if run_resnet:
    train_losses_resnet = []
    val_losses_resnet = []
    models_resnet = []
    for epoch in range(n_epochs):
        train_loss, train_acc = train_step(model_resnet18,
                                        optimizer_resnet18,
                                        criterion_resnet18,
                                        train_loader)
        val_loss, val_acc = val_step(model_resnet18,
                                    criterion_resnet18,
                                    test_loader)
        train_losses_resnet.append(train_loss)
        val_losses_resnet.append(val_loss)
        models_resnet += [copy.deepcopy(model_resnet18)]

        print_step_report(epoch=epoch,
                        train_loss=train_loss,
                        val_loss=val_loss,
                        train_acc=train_acc,
                        val_acc=val_acc)

    torch.save(models_resnet, "resnet_models.pth")

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy Model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

if run_toy_model:
    rlct_estimates_sgnht = []
    rlct_estimates_sgld = []
    for model in models_toy:
        rlct_estimate_sgnht = estimate_learning_coeff(
            model,
            train_loader,
            criterion=criterion_toy,
            optimizer_kwargs=dict(
                lr=1e-7,
                diffusion_factor=0.01,
                num_samples=len(train_data),
            ),
            sampling_method=SGNHT,
            num_chains=1,
            num_draws=400,
            num_burnin_steps=0,
            num_steps_bw_draws=1,
            device=DEVICE,
        )
        rlct_estimate_sgld = estimate_learning_coeff(
            model,
            train_loader,
            criterion=criterion_toy,
            optimizer_kwargs=dict(
                lr=1e-5,
                noise_level=1.0,
                elasticity=100.0,
                num_samples=len(train_data),
                temperature="adaptive",
            ),
            sampling_method=SGLD,
            num_chains=1,
            num_draws=400,
            num_burnin_steps=0,
            num_steps_bw_draws=1,
            device=DEVICE,
        )
        rlct_estimates_sgnht += [rlct_estimate_sgnht]
        rlct_estimates_sgld += [rlct_estimate_sgld]
        print(rlct_estimate_sgld, rlct_estimate_sgnht)

    # plotting
    plot_lambda_vs_epochs(train_losses_toy, val_losses_toy, rlct_estimates_sgnht, rlct_estimates_sgld)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# ______________________    Resnet18 models    ______________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

if run_resnet:
    rlct_estimates_sgnht = []
    rlct_estimates_sgld = []
    for model in models_resnet:
        rlct_estimate_sgnht = estimate_learning_coeff(
            model,
            train_loader,
            criterion=criterion_resnet18,
            optimizer_kwargs=dict(
                lr=1e-7,
                diffusion_factor=0.01,
                num_samples=len(train_data),
            ),
            sampling_method=SGNHT,
            num_chains=1,
            num_draws=400,
            num_burnin_steps=0,
            num_steps_bw_draws=1,
            device=DEVICE,
        )
        rlct_estimate_sgld = estimate_learning_coeff(
            model,
            train_loader,
            criterion=criterion_resnet18,
            optimizer_kwargs=dict(
                lr=1e-5,
                noise_level=1.0,
                elasticity=100.0,
                num_samples=len(train_data),
                temperature="adaptive",
            ),
            sampling_method=SGLD,
            num_chains=1,
            num_draws=400,
            num_burnin_steps=0,
            num_steps_bw_draws=1,
            device=DEVICE,
        )
        rlct_estimates_sgnht += [rlct_estimate_sgnht]
        rlct_estimates_sgld += [rlct_estimate_sgld]
        print(rlct_estimate_sgld, rlct_estimate_sgnht)

    # plotting
    plot_lambda_vs_epochs(train_losses_resnet, val_losses_resnet, rlct_estimates_sgnht, rlct_estimates_sgld)

### Settings and runs label sets $Card(\mathcal{L}abels)=2$

The different sets of labels to explore are: $\{1, other\}$ ; $\{2, other\}$ ; $\{4, other\}$ ; $\{5, other\}$ ; $\{9, other\}$.
`other` cannot be put as a label as it, it needs to be encoded into a integer. In order to relabel as we wish, we simply map the first label/digit of the sets to `1` and all the other to `0`. For example for the set $\{4,other\}$ it means that the images representing `4` will be relabeled as `1` and the images representing any of $[0, 1, 2, 3, 5, 6, 7, 8, 9]$ will be relabeled as `0`.

* labels modification and data loader
* Run for the toy model
* Run for the resnet18 model

In [ ]:
# preparing the output directories of XP1
dirs = ["XP1",
        "XP1/card_2-case_1",
        "XP1/card_2-case_2","XP1/card_2-case_3", "XP1/card_2-case_4", "XP1/card_2-case_5",
        "XP1/card_3-case_1", "XP1/card_3-case_2","XP1/card_3-case_3", "XP1/card_3-case_4", "XP1/card_3-case_5",
        "XP1/card_4-case_1", "XP1/card_4-case_2","XP1/card_4-case_3", "XP1/card_4-case_4",
        "XP1/card_5-case_1", "XP1/card_5-case_2","XP1/card_5-case_3", "XP1/card_5-case_4",
        "XP1/card_6-case_1", "XP1/card_6-case_2","XP1/card_6-case_3", "XP1/card_6-case_4",
        "XP1/card_8-case_1", "XP1/card_8-case_2",
        "XP1/card_9-case_1", "XP1/card_9-case_2",
        "XP1/card_10-case_1",
        ]

for path in dirs:
        os.mkdir(path)

In [ ]:
def load_and_process_data(bs: int, mapping_label_to_encode: dict) -> tuple[DataLoader, DataLoader]:
    """ Loads MNIST dataset, relabel the images of train and test sets

    Function load MNIST dataset and performs the relabelling. The loading performs the transformation
    necessary to normalize the data distribution.
    Before forging the data loaders, the operation of relabelling to execute the experiments
    is performed.
    Arguments:
        bs: batch size, excepting a positive integer
        mapping_label_to_encode: mapping between original labels and target labels

    Returns:
        train_loader, test_loader: data loader of train and test
    """
    transform = transforms.Compose([transforms.ToTensor(),
                                    transforms.Normalize((0.1307,), (0.3081,))])
    # Load train data
    train_data = datasets.MNIST("../data", train=True, transform=transform, download=True)
    for old_label, new_label in mapping_label_to_encode.items():
        train_data.targets[train_data.targets == old_label] = new_label
    print("set of labels in train data: ", train_data.targets.unique())
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

    # Load test data
    test_data = datasets.MNIST("../data", train=False, transform=transform)
    for old_label, new_label in mapping_label_to_encode.items():
        test_data.targets[test_data.targets == old_label] = new_label
    print("set of labels in test data: ", test_data.targets.unique())
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

    return train_loader, test_loader

def models_factory(model_type: Literal["toy_model", "resnet18"],
                   optim_cfg: dict,
                   optim_name: Literal['SGD'] = "SGD",
                   criterion_name: Literal["CrossEntropyLoss"]="CrossEntropyLoss",
                   lr: confloat(gt=0, lt=1)=0.005,
                   n_epochs: conint(gt=0)=20,
                   nb_classes: conint(ge=2, le=10)=10
                   ):
    """Very light factory to handle the model instanciation.

    The main purpose of this method is to centralize the models instance
    allowing lighter code cells in the following.

    Note:
        versatility is very limited here, but the idea is to use a common interface to
        generate a model withing a collection of possibility

    Args:
        model_type: name of the model to return
        optim_name: optimiser name of the model
        optim_cfg: configuration of the optimiser
        criterion_name: Loss function associated with the model for training.
            Defaults to "CrossEntropyLoss".
        lr: learning rate. Defaults to 0.005.
        n_epochs: number of epochs. Defaults to 20.
        nb_classes: number of classes of the task.
    """
    if model_type == "toy_model":
        model = ToyNet(output_dim=nb_classes).to(DEVICE)
    elif model_type == "resnet18":
        model = resnet18(weights=None).to(DEVICE)

        ## changing the last fully connected layer of N neurons, each corresponding to a class.
        new_fc_act = nn.Linear(model.fc.in_features, nb_classes)
        model.fc = new_fc_act

        ## Changing the first conv2d layer to match the input image shape
        model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

    else:
        raise ValueError(f"Model factory does not handle model_type: '{model_type}'")

    # Loss definition
    if criterion_name == "CrossEntropyLoss":
        criterion = nn.CrossEntropyLoss()
    else:
        ValueError("Model factory only handles Cross Entropy Loss.")

    # Optimizer definition
    if optim_name == "SGD":
        optimizer = optim.SGD(model.parameters(), lr=lr, **optim_cfg)
    else:
        ValueError("Model factory only handles Cross Entropy Loss.")

    return model, optimizer, criterion


def export_loss_acc_to_csv(path: str, train_loss: list[float], test_loss: list[float], train_acc: list[float], test_acc: list[float]):
    """Save losses and accuracy into csv

    Args:
        path: path to file to save the data
        train_loss: list of the train loss along epochs
        test_loss: list of the test loss along epochs
        train_acc: list of the train accuracy along epochs
        test_acc: list of the test accuracy along epochs
    """
    df = pd.DataFrame().from_dict({"train_loss": train_loss,
                                   "test_loss": test_loss,
                                   "train_accuracy": train_acc,
                                   "test_accuracy": test_acc})
    df.to_csv(path)

def export_rlct_to_csv(path: str, rlct_estimates_sgnht: list[float], rlct_estimates_sgld: list[float]):
    """Save losses and accuracy into csv

    Args:
        path: path to file to save the data
        rlct_estimates_sgnht: list of rlct estimated with Stochastic Gradient Nose-Hoover Thermostat
        rlct_estimates_sgld: list of rlct estimated with Stochastic Gradient Langevin Dynamics
    """
    df = pd.DataFrame().from_dict({"rlct_estimates_sgnht": rlct_estimates_sgnht,
                                   "rlct_estimates_sgld": rlct_estimates_sgld})
    df.to_csv(path)

In [ ]:
# keys are the original label, values are the new labels
map_label_card_2_1 = {1: 1, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0}
map_label_card_2_2 = {1: 0, 2: 1, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0}
map_label_card_2_3 = {1: 0, 2: 0, 3: 0, 4: 1, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0}
map_label_card_2_4 = {1: 0, 2: 0, 3: 0, 4: 0, 5: 1, 6: 0, 7: 0, 8: 0, 9: 0}
map_label_card_2_5 = {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 1}

##### 1.1 XP1 - labels set $\mathcal{Card} = 2$ - $\mathcal{L}abels=\{1, other\}$

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# ________________________    Data Loader    ________________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
## definition of the transformation to apply to dataset


## Load train data
batch_size = 512

train_loader, test_loader = load_and_process_data(bs=batch_size,
                                                  mapping_label_to_encode=map_label_card_2_1)

# ------------------------------------------------------
n_classes = 2
n_epochs = 20
lr = 0.005
dict_optim_cfg = {"momentum":0.9 ,"nesterov": True}
# ------------------------------------------------------
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

if run_toy_model:
    model_toy, optimizer_toy, criterion_toy = models_factory(model_type="toy_model",
                                                            optim_cfg=dict_optim_cfg,
                                                            nb_classes=n_classes)

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 instance    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# Initialize model, loss, optimizer and sgld sampler
## taking an untrained resnet18
if run_resnet:
    models_resnet, optimizer_resnet, criterion_resnet = models_factory(model_type="toy_model",
                                                                    optim_cfg=dict_optim_cfg,
                                                                    nb_classes=n_classes)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 training    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

if run_resnet:
    train_losses_resnet_xp1_1 = []
    val_losses_resnet_xp1_1 = []
    train_acc_resnet_xp1_1 = []
    val_acc_resnet_xp1_1 = []
    models_resnet_xp1_1 = []
    for epoch in range(n_epochs):
        train_loss, train_acc = train_step(model_resnet18,
                                        optimizer_resnet18,
                                        criterion_resnet18,
                                        train_loader)
        val_loss, val_acc = val_step(model_resnet18,
                                    criterion_resnet18,
                                    test_loader)
        train_losses_resnet_xp1_1.append(train_loss)
        val_losses_resnet_xp1_1.append(val_loss)
        train_acc_resnet_xp1_1.append(train_acc)
        val_acc_resnet_xp1_1.append(val_acc)
        models_resnet_xp1_1 += [copy.deepcopy(model_resnet18)]

        print_step_report(epoch=epoch,
                        train_loss=train_loss,
                        val_loss=val_loss,
                        train_acc=train_acc,
                        val_acc=val_acc)

    torch.save(models_resnet_xp1_1, "XP1/card_2-case_1/resnet_models.pth")
    export_loss_acc_to_csv(path="XP1/card_2-case_1/resnet_loss_acc.csv",
                       train_loss=train_losses_resnet_xp1_1,
                       test_loss=val_losses_resnet_xp1_1,
                       train_acc=train_acc_resnet_xp1_1,
                       test_acc=val_acc_resnet_xp1_1)


# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model training    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
if run_toy_model:
    train_losses_toy_xp1_1 = []
    val_losses_toy_xp1_1 = []
    train_acc_toy_xp1_1 = []
    val_acc_toy_xp1_1 = []
    models_toy_xp1_1 = []
    for epoch in range(n_epochs):
        train_loss, train_acc = train_step(model_toy,
                                        optimizer_toy,
                                        criterion_toy,
                                        train_loader)
        val_loss, val_acc = val_step(model_toy,
                                    criterion_toy,
                                    test_loader)
        train_losses_toy_xp1_1.append(train_loss)
        val_losses_toy_xp1_1.append(val_loss)
        train_acc_toy_xp1_1.append(train_acc)
        val_acc_toy_xp1_1.append(val_acc)
        models_toy_xp1_1 += [copy.deepcopy(model_toy)]

        print_step_report(epoch=epoch,
                        train_loss=train_loss,
                        val_loss=val_loss,
                        train_acc=train_acc,
                        val_acc=val_acc)

    torch.save(models_toy_xp1_1, "XP1/card_2-case_1/toy_models.pth")
    export_loss_acc_to_csv(path="XP1/card_2-case_1/toy_loss_acc.csv",
                       train_loss=train_losses_toy_xp1_1,
                       test_loss=val_losses_toy_xp1_1,
                       train_acc=train_acc_toy_xp1_1,
                       test_acc=val_acc_toy_xp1_1)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy Model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

rlct_estimates_sgnht = []
rlct_estimates_sgld = []
for model in models_toy_xp1_1:
    rlct_estimate_sgnht = estimate_learning_coeff(
        model,
        train_loader,
        criterion=criterion_toy,
        optimizer_kwargs=dict(
            lr=1e-7,
            diffusion_factor=0.01,
            num_samples=len(train_data),
        ),
        sampling_method=SGNHT,
        num_chains=1,
        num_draws=400,
        num_burnin_steps=0,
        num_steps_bw_draws=1,
        device=DEVICE,
    )
    rlct_estimate_sgld = estimate_learning_coeff(
        model,
        train_loader,
        criterion=criterion_toy,
        optimizer_kwargs=dict(
            lr=1e-5,
            noise_level=1.0,
            elasticity=100.0,
            num_samples=len(train_data),
            temperature="adaptive",
        ),
        sampling_method=SGLD,
        num_chains=1,
        num_draws=400,
        num_burnin_steps=0,
        num_steps_bw_draws=1,
        device=DEVICE,
    )
    rlct_estimates_sgnht += [rlct_estimate_sgnht]
    rlct_estimates_sgld += [rlct_estimate_sgld]
    print(rlct_estimate_sgld, rlct_estimate_sgnht)

# plotting
export_rlct_to_csv("XP1/card_2-case_1/toy_rlct_estimated.csv",
                    rlct_estimates_sgnht=rlct_estimates_sgnht,
                    rlct_estimates_sgld=rlct_estimates_sgld)
plot_lambda_vs_epochs(train_losses_toy_xp1_1, val_losses_toy_xp1_1, rlct_estimates_sgnht, rlct_estimates_sgld)


In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# ______________________    Resnet18 instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

rlct_estimates_sgnht = []
rlct_estimates_sgld = []
for model in models_resnet_xp1_1:
    rlct_estimate_sgnht = estimate_learning_coeff(
        model,
        train_loader,
        criterion=criterion_resnet,
        optimizer_kwargs=dict(
            lr=1e-7,
            diffusion_factor=0.01,
            num_samples=len(train_data),
        ),
        sampling_method=SGNHT,
        num_chains=1,
        num_draws=400,
        num_burnin_steps=0,
        num_steps_bw_draws=1,
        device=DEVICE,
    )
    rlct_estimate_sgld = estimate_learning_coeff(
        model,
        train_loader,
        criterion=criterion_resnet,
        optimizer_kwargs=dict(
            lr=1e-5,
            noise_level=1.0,
            elasticity=100.0,
            num_samples=len(train_data),
            temperature="adaptive",
        ),
        sampling_method=SGLD,
        num_chains=1,
        num_draws=400,
        num_burnin_steps=0,
        num_steps_bw_draws=1,
        device=DEVICE,
    )
    rlct_estimates_sgnht += [rlct_estimate_sgnht]
    rlct_estimates_sgld += [rlct_estimate_sgld]
    print(rlct_estimate_sgld, rlct_estimate_sgnht)

# plotting
export_rlct_to_csv("XP1/card_2-case_1/resnet_rlct_estimated.csv",
                    rlct_estimates_sgnht=rlct_estimates_sgnht,
                    rlct_estimates_sgld=rlct_estimates_sgld)
plot_lambda_vs_epochs(train_losses_resnet_xp1_1, val_losses_resnet_xp1_1, rlct_estimates_sgnht, rlct_estimates_sgld)

##### 1.2 XP1 - labels set $\mathcal{Card} = 2$ - $\mathcal{L}abels=\{2, other\}$

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# ________________________    Data Loader    ________________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
## definition of the transformation to apply to dataset


## Load train data
batch_size = 512

train_loader, test_loader = load_and_process_data(bs=batch_size,
                                                  mapping_label_to_encode=map_label_card_2_2)

# ---------------------------------------------------
n_classes = 2
n_epochs = 20
lr = 0.005
dict_optim_cfg = {"momentum":0.9 ,"nesterov": True}
# ---------------------------------------------------
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
if run_toy_model:
    model_toy, optimizer_toy, criterion_toy = models_factory(model_type="toy_model",
                                                            optim_cfg=dict_optim_cfg,
                                                            nb_classes=n_classes)

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 instance    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# Initialize model, loss, optimizer and sgld sampler
## taking an untrained resnet18
if run_resnet:
    models_resnet, optimizer_resnet, criterion_resnet = models_factory(model_type="toy_model",
                                                                    optim_cfg=dict_optim_cfg,
                                                                    nb_classes=n_classes)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 training    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

if run_resnet:
    train_losses_resnet_xp1_2 = []
    val_losses_resnet_xp1_2 = []
    train_acc_resnet_xp1_2 = []
    val_acc_resnet_xp1_2 = []
    models_resnet_xp1_2 = []
    for epoch in range(n_epochs):
        train_loss, train_acc = train_step(model_resnet18,
                                        optimizer_resnet18,
                                        criterion_resnet18,
                                        train_loader)
        val_loss, val_acc = val_step(model_resnet18,
                                    criterion_resnet18,
                                    test_loader)
        train_losses_resnet_xp1_2.append(train_loss)
        val_losses_resnet_xp1_2.append(val_loss)
        models_resnet_xp1_2 += [copy.deepcopy(model_resnet18)]

        print_step_report(epoch=epoch,
                        train_loss=train_loss,
                        val_loss=val_loss,
                        train_acc=train_acc,
                        val_acc=val_acc)

    torch.save(models_resnet_xp1_2, "XP1/card_2-case_2/resnet_models.pth")
    export_loss_acc_to_csv(path="XP1/card_2-case_2/resnet_loss_acc.csv",
                           train_loss=train_losses_resnet_xp1_2,
                           test_loss=val_losses_resnet_xp1_2,
                           train_acc=train_acc_resnet_xp1_2,
                           test_acc=val_acc_resnet_xp1_2)

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model training    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
if run_toy_model:
    train_losses_toy_xp1_2 = []
    val_losses_toy_xp1_2 = []
    train_acc_toy_xp1_2 = []
    val_acc_toy_xp1_2 = []
    models_toy_xp1_2 = []
    for epoch in range(n_epochs):
        train_loss, train_acc = train_step(model_toy,
                                        optimizer_toy,
                                        criterion_toy,
                                        train_loader)
        val_loss, val_acc = val_step(model_toy,
                                    criterion_toy,
                                    test_loader)
        train_losses_toy_xp1_2.append(train_loss)
        val_losses_toy_xp1_2.append(val_loss)
        train_acc_toy_xp1_2.append(train_acc)
        val_acc_toy_xp1_2.append(val_acc)
        models_toy_xp1_2 += [copy.deepcopy(model_toy)]

        print_step_report(epoch=epoch,
                        train_loss=train_loss,
                        val_loss=val_loss,
                        train_acc=train_acc,
                        val_acc=val_acc)

    torch.save(models_toy_xp1_2, "XP1/card_2-case_2/toy_models.pth")
    export_loss_acc_to_csv(path="XP1/card_2-case_2/toy_loss_acc.csv",
                           train_loss=train_losses_toy_xp1_2,
                           test_loss=val_losses_toy_xp1_2,
                           train_acc=train_acc_toy_xp1_2,
                           test_acc=val_acc_toy_xp1_2)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy Model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

rlct_estimates_sgnht = []
rlct_estimates_sgld = []
for model in models_toy_xp1_2:
    rlct_estimate_sgnht = estimate_learning_coeff(
        model,
        train_loader,
        criterion=criterion_toy,
        optimizer_kwargs=dict(
            lr=1e-7,
            diffusion_factor=0.01,
            num_samples=len(train_data),
        ),
        sampling_method=SGNHT,
        num_chains=1,
        num_draws=400,
        num_burnin_steps=0,
        num_steps_bw_draws=1,
        device=DEVICE,
    )
    rlct_estimate_sgld = estimate_learning_coeff(
        model,
        train_loader,
        criterion=criterion_toy,
        optimizer_kwargs=dict(
            lr=1e-5,
            noise_level=1.0,
            elasticity=100.0,
            num_samples=len(train_data),
            temperature="adaptive",
        ),
        sampling_method=SGLD,
        num_chains=1,
        num_draws=400,
        num_burnin_steps=0,
        num_steps_bw_draws=1,
        device=DEVICE,
    )
    rlct_estimates_sgnht += [rlct_estimate_sgnht]
    rlct_estimates_sgld += [rlct_estimate_sgld]
    print(rlct_estimate_sgld, rlct_estimate_sgnht)

# plotting
export_rlct_to_csv("XP1/card_2-case_2/toy_rlct_estimated.csv",
                    rlct_estimates_sgnht=rlct_estimates_sgnht,
                    rlct_estimates_sgld=rlct_estimates_sgld)
plot_lambda_vs_epochs(train_losses_toy_xp1_2, val_losses_toy_xp1_2, rlct_estimates_sgnht, rlct_estimates_sgld)


##### 1.3 XP1 - labels set $\mathcal{Card} = 2$ - $\mathcal{L}abels=\{4, other\}$

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# ________________________    Data Loader    ________________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
## definition of the transformation to apply to dataset


## Load train data
batch_size = 512

train_loader, test_loader = load_and_process_data(bs=batch_size,
                                                  mapping_label_to_encode=map_label_card_2_3)

# -------------------------------------------------------
n_classes = 2
n_epochs = 20
lr = 0.005
dict_optim_cfg = {"momentum":0.9 ,"nesterov": True}
# -------------------------------------------------------
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
if run_toy_model:
    model_toy, optimizer_toy, criterion_toy = models_factory(model_type="toy_model",
                                                            optim_cfg=dict_optim_cfg,
                                                            nb_classes=n_classes)

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 instance    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# Initialize model, loss, optimizer and sgld sampler
## taking an untrained resnet18
if run_resnet:
    models_resnet, optimizer_resnet, criterion_resnet = models_factory(model_type="toy_model",
                                                                    optim_cfg=dict_optim_cfg,
                                                                    nb_classes=n_classes)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 training    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

if run_resnet:
    train_losses_resnet_xp1_3 = []
    val_losses_resnet_xp1_3 = []
    train_acc_resnet_xp1_3 = []
    val_acc_resnet_xp1_3 = []
    models_resnet_xp1_3 = []
    for epoch in range(n_epochs):
        train_loss, train_acc = train_step(model_resnet18,
                                        optimizer_resnet18,
                                        criterion_resnet18,
                                        train_loader)
        val_loss, val_acc = val_step(model_resnet18,
                                    criterion_resnet18,
                                    test_loader)
        train_losses_resnet_xp1_3.append(train_loss)
        val_losses_resnet_xp1_3.append(val_loss)
        train_acc_resnet_xp1_3.append(train_acc)
        val_acc_resnet_xp1_3.append(val_acc)
        models_resnet_xp1_3 += [copy.deepcopy(model_resnet18)]

        print_step_report(epoch=epoch,
                        train_loss=train_loss,
                        val_loss=val_loss,
                        train_acc=train_acc,
                        val_acc=val_acc)

    torch.save(models_resnet_xp1_3, "XP1/card_2-case_3/resnet_models.pth")
    export_loss_acc_to_csv(path="XP1/card_2-case_3/resnet_loss_acc.csv",
                           train_loss=train_losses_resnet_xp1_3,
                           test_loss=val_losses_resnet_xp1_3,
                           train_acc=train_acc_resnet_xp1_3,
                           test_acc=val_acc_resnet_xp1_3)

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model training    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
if run_toy_model:
    train_losses_toy_xp1_3 = []
    val_losses_toy_xp1_3 = []
    train_acc_toy_xp1_3 = []
    val_acc_toy_xp1_3 = []
    models_toy_xp1_3 = []
    for epoch in range(n_epochs):
        train_loss, train_acc = train_step(model_toy,
                                        optimizer_toy,
                                        criterion_toy,
                                        train_loader)
        val_loss, val_acc = val_step(model_toy,
                                    criterion_toy,
                                    test_loader)
        train_losses_toy_xp1_3.append(train_loss)
        val_losses_toy_xp1_3.append(val_loss)
        train_acc_toy_xp1_3.append(train_acc)
        val_acc_toy_xp1_3.append(val_acc)
        models_toy_xp1_3 += [copy.deepcopy(model_toy)]

        print_step_report(epoch=epoch,
                        train_loss=train_loss,
                        val_loss=val_loss,
                        train_acc=train_acc,
                        val_acc=val_acc)

    torch.save(models_toy_xp1_3, "XP1/card_2-case_3/toy_models.pth")
    export_loss_acc_to_csv(path="XP1/card_2-case_3/toy_loss_acc.csv",
                           train_loss=train_losses_toy_xp1_3,
                           test_loss=val_losses_toy_xp1_3,
                           train_acc=train_acc_toy_xp1_3,
                           test_acc=val_acc_toy_xp1_3)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy Model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

rlct_estimates_sgnht = []
rlct_estimates_sgld = []
for model in models_toy_xp1_3:
    rlct_estimate_sgnht = estimate_learning_coeff(
        model,
        train_loader,
        criterion=criterion_toy,
        optimizer_kwargs=dict(
            lr=1e-7,
            diffusion_factor=0.01,
            num_samples=len(train_data),
        ),
        sampling_method=SGNHT,
        num_chains=1,
        num_draws=400,
        num_burnin_steps=0,
        num_steps_bw_draws=1,
        device=DEVICE,
    )
    rlct_estimate_sgld = estimate_learning_coeff(
        model,
        train_loader,
        criterion=criterion_toy,
        optimizer_kwargs=dict(
            lr=1e-5,
            noise_level=1.0,
            elasticity=100.0,
            num_samples=len(train_data),
            temperature="adaptive",
        ),
        sampling_method=SGLD,
        num_chains=1,
        num_draws=400,
        num_burnin_steps=0,
        num_steps_bw_draws=1,
        device=DEVICE,
    )
    rlct_estimates_sgnht += [rlct_estimate_sgnht]
    rlct_estimates_sgld += [rlct_estimate_sgld]
    print(rlct_estimate_sgld, rlct_estimate_sgnht)

# plotting
export_rlct_to_csv("XP1/card_2-case_3/toy_rlct_estimated.csv",
                    rlct_estimates_sgnht=rlct_estimates_sgnht,
                    rlct_estimates_sgld=rlct_estimates_sgld)
plot_lambda_vs_epochs(train_losses_toy_xp1_3, val_losses_toy_xp1_3, rlct_estimates_sgnht, rlct_estimates_sgld)


##### 1.4 XP1 - labels set $\mathcal{Card} = 2$ - $\mathcal{L}abels=\{5, other\}$

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# ________________________    Data Loader    ________________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
## definition of the transformation to apply to dataset


## Load train data
batch_size = 512

train_loader, test_loader = load_and_process_data(bs=batch_size,
                                                  mapping_label_to_encode=map_label_card_2_4)
# ---------------------------------------------------
n_classes = 2
n_epochs = 20
lr = 0.005
dict_optim_cfg = {"momentum":0.9 ,"nesterov": True}
# ---------------------------------------------------
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
if run_toy_model:
    model_toy, optimizer_toy, criterion_toy = models_factory(model_type="toy_model",
                                                            optim_cfg=dict_optim_cfg,
                                                            nb_classes=n_classes)

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 instance    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# Initialize model, loss, optimizer and sgld sampler
## taking an untrained resnet18
if run_resnet:
    models_resnet, optimizer_resnet, criterion_resnet = models_factory(model_type="toy_model",
                                                                    optim_cfg=dict_optim_cfg,
                                                                    nb_classes=n_classes)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 training    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

if run_resnet:
    train_losses_resnet_xp1_4 = []
    val_losses_resnet_xp1_4 = []
    train_acc_resnet_xp1_4 = []
    val_acc_resnet_xp1_4 = []
    models_resnet_xp1_4 = []
    for epoch in range(n_epochs):
        train_loss, train_acc = train_step(model_resnet18,
                                        optimizer_resnet18,
                                        criterion_resnet18,
                                        train_loader)
        val_loss, val_acc = val_step(model_resnet18,
                                    criterion_resnet18,
                                    test_loader)
        train_losses_resnet_xp1_4.append(train_loss)
        val_losses_resnet_xp1_4.append(val_loss)
        train_acc_resnet_xp1_4.append(train_acc)
        val_acc_resnet_xp1_4.append(val_acc)
        models_resnet_xp1_4 += [copy.deepcopy(model_resnet18)]

        print_step_report(epoch=epoch,
                        train_loss=train_loss,
                        val_loss=val_loss,
                        train_acc=train_acc,
                        val_acc=val_acc)

    torch.save(models_resnet_xp1_4, "XP1/card_2-case_4/resnet_models.pth")
    export_loss_acc_to_csv(path="XP1/card_2-case_4/resnet_loss_acc.csv",
                           train_loss=train_losses_resnet_xp1_4,
                           test_loss=val_losses_resnet_xp1_4,
                           train_acc=train_acc_resnet_xp1_4,
                           test_acc=val_acc_resnet_xp1_4)

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model training    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
if run_toy_model:
    train_losses_toy_xp1_4 = []
    val_losses_toy_xp1_4 = []
    train_acc_toy_xp1_4 = []
    val_acc_toy_xp1_4 = []
    models_toy_xp1_4 = []
    for epoch in range(n_epochs):
        train_loss, train_acc = train_step(model_toy,
                                        optimizer_toy,
                                        criterion_toy,
                                        train_loader)
        val_loss, val_acc = val_step(model_toy,
                                    criterion_toy,
                                    test_loader)
        train_losses_toy_xp1_4.append(train_loss)
        val_losses_toy_xp1_4.append(val_loss)
        train_acc_toy_xp1_4.append(train_acc)
        val_acc_toy_xp1_4.append(val_acc)
        models_toy_xp1_4 += [copy.deepcopy(model_toy)]

        print_step_report(epoch=epoch,
                        train_loss=train_loss,
                        val_loss=val_loss,
                        train_acc=train_acc,
                        val_acc=val_acc)

    torch.save(models_toy_xp1_4, "XP1/card_2-case_4/toy_models.pth")
    export_loss_acc_to_csv(path="XP1/card_2-case_4/toy_loss_acc.csv",
                           train_loss=train_losses_toy_xp1_4,
                           test_loss=val_losses_toy_xp1_4,
                           train_acc=train_acc_toy_xp1_4,
                           test_acc=val_acc_toy_xp1_4)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy Model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

rlct_estimates_sgnht = []
rlct_estimates_sgld = []
for model in models_toy_xp1_4:
    rlct_estimate_sgnht = estimate_learning_coeff(
        model,
        train_loader,
        criterion=criterion_toy,
        optimizer_kwargs=dict(
            lr=1e-7,
            diffusion_factor=0.01,
            num_samples=len(train_data),
        ),
        sampling_method=SGNHT,
        num_chains=1,
        num_draws=400,
        num_burnin_steps=0,
        num_steps_bw_draws=1,
        device=DEVICE,
    )
    rlct_estimate_sgld = estimate_learning_coeff(
        model,
        train_loader,
        criterion=criterion_toy,
        optimizer_kwargs=dict(
            lr=1e-5,
            noise_level=1.0,
            elasticity=100.0,
            num_samples=len(train_data),
            temperature="adaptive",
        ),
        sampling_method=SGLD,
        num_chains=1,
        num_draws=400,
        num_burnin_steps=0,
        num_steps_bw_draws=1,
        device=DEVICE,
    )
    rlct_estimates_sgnht += [rlct_estimate_sgnht]
    rlct_estimates_sgld += [rlct_estimate_sgld]
    print(rlct_estimate_sgld, rlct_estimate_sgnht)

# plotting
export_rlct_to_csv("XP1/card_2-case_4/toy_rlct_estimated.csv",
                    rlct_estimates_sgnht=rlct_estimates_sgnht,
                    rlct_estimates_sgld=rlct_estimates_sgld)
plot_lambda_vs_epochs(train_losses_toy_xp1_4, val_losses_toy_xp1_4, rlct_estimates_sgnht, rlct_estimates_sgld)


##### 1.5 XP1 - labels set $\mathcal{Card} = 2$ - $\mathcal{L}abels=\{9, other\}$

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# ________________________    Data Loader    ________________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
## definition of the transformation to apply to dataset


## Load train data
batch_size = 512

train_loader, test_loader = load_and_process_data(bs=batch_size,
                                                  mapping_label_to_encode=map_label_card_2_5)
# --------------------------------------------------
n_classes = 2
n_epochs = 20
lr = 0.005
dict_optim_cfg = {"momentum":0.9 ,"nesterov": True}
# --------------------------------------------------
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
if run_toy_model:
    model_toy, optimizer_toy, criterion_toy = models_factory(model_type="toy_model",
                                                            optim_cfg=dict_optim_cfg,
                                                            nb_classes=n_classes)

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 instance    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# Initialize model, loss, optimizer and sgld sampler
## taking an untrained resnet18
if run_resnet:
    models_resnet, optimizer_resnet, criterion_resnet = models_factory(model_type="toy_model",
                                                                    optim_cfg=dict_optim_cfg,
                                                                    nb_classes=n_classes)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 training    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

if run_resnet:
    train_losses_resnet_xp1_5 = []
    val_losses_resnet_xp1_5 = []
    train_acc_resnet_xp1_5 = []
    val_acc_resnet_xp1_5 = []
    models_resnet_xp1_5 = []
    for epoch in range(n_epochs):
        train_loss, train_acc = train_step(model_resnet18,
                                        optimizer_resnet18,
                                        criterion_resnet18,
                                        train_loader)
        val_loss, val_acc = val_step(model_resnet18,
                                    criterion_resnet18,
                                    test_loader)
        train_losses_resnet_xp1_5.append(train_loss)
        val_losses_resnet_xp1_5.append(val_loss)
        train_acc_resnet_xp1_5.append(train_acc)
        val_acc_resnet_xp1_5.append(val_acc)
        models_resnet_xp1_5 += [copy.deepcopy(model_resnet18)]

        print_step_report(epoch=epoch,
                        train_loss=train_loss,
                        val_loss=val_loss,
                        train_acc=train_acc,
                        val_acc=val_acc)

    torch.save(models_resnet_xp1_5, "XP1/card_2-case_5/resnet_models.pth")
    export_loss_acc_to_csv(path="XP1/card_2-case_5/resnet_loss_acc.csv",
                           train_loss=train_losses_resnet_xp1_5,
                           test_loss=val_losses_resnet_xp1_5,
                           train_acc=train_acc_resnet_xp1_5,
                           test_acc=val_acc_resnet_xp1_5)

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model training    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
if run_toy_model:
    train_losses_toy_xp1_5 = []
    val_losses_toy_xp1_5 = []
    train_acc_toy_xp1_5 = []
    val_acc_toy_xp1_5 = []
    models_toy_xp1_5 = []
    for epoch in range(n_epochs):
        train_loss, train_acc = train_step(model_toy,
                                        optimizer_toy,
                                        criterion_toy,
                                        train_loader)
        val_loss, val_acc = val_step(model_toy,
                                    criterion_toy,
                                    test_loader)
        train_losses_toy_xp1_5.append(train_loss)
        val_losses_toy_xp1_5.append(val_loss)
        train_acc_toy_xp1_5.append(train_acc)
        val_acc_toy_xp1_5.append(val_acc)
        models_toy_xp1_5 += [copy.deepcopy(model_toy)]

        print_step_report(epoch=epoch,
                        train_loss=train_loss,
                        val_loss=val_loss,
                        train_acc=train_acc,
                        val_acc=val_acc)

    torch.save(models_toy_xp1_5, "XP1/card_2-case_5/toy_models.pth")
    export_loss_acc_to_csv(path="XP1/card_2-case_5/toy_loss_acc.csv",
                            train_loss=train_losses_toy_xp1_5,
                            test_loss=val_losses_toy_xp1_5,
                            train_acc=train_acc_toy_xp1_5,
                            test_acc=val_acc_toy_xp1_5)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy Model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

rlct_estimates_sgnht = []
rlct_estimates_sgld = []
for model in models_toy_xp1_5:
    rlct_estimate_sgnht = estimate_learning_coeff(
        model,
        train_loader,
        criterion=criterion_toy,
        optimizer_kwargs=dict(
            lr=1e-7,
            diffusion_factor=0.01,
            num_samples=len(train_data),
        ),
        sampling_method=SGNHT,
        num_chains=1,
        num_draws=400,
        num_burnin_steps=0,
        num_steps_bw_draws=1,
        device=DEVICE,
    )
    rlct_estimate_sgld = estimate_learning_coeff(
        model,
        train_loader,
        criterion=criterion_toy,
        optimizer_kwargs=dict(
            lr=1e-5,
            noise_level=1.0,
            elasticity=100.0,
            num_samples=len(train_data),
            temperature="adaptive",
        ),
        sampling_method=SGLD,
        num_chains=1,
        num_draws=400,
        num_burnin_steps=0,
        num_steps_bw_draws=1,
        device=DEVICE,
    )
    rlct_estimates_sgnht += [rlct_estimate_sgnht]
    rlct_estimates_sgld += [rlct_estimate_sgld]
    print(rlct_estimate_sgld, rlct_estimate_sgnht)

# plotting
export_rlct_to_csv("XP1/card_2-case_5/toy_rlct_estimated.csv",
                    rlct_estimates_sgnht=rlct_estimates_sgnht,
                    rlct_estimates_sgld=rlct_estimates_sgld)
plot_lambda_vs_epochs(train_losses_toy_xp1_5, val_losses_toy_xp1_5, rlct_estimates_sgnht, rlct_estimates_sgld)


### Settings and runs label sets $Card(\mathcal{L}abels)=3$

* Labels modification and data loader
* Toy model for 3 classes
* Resnet18 model for 3 classes

In [ ]:
# keys are the original label, values are the new labels
# [{2, 6, other} ; {3, 6, other} ; {0, 4, other} ; {8, 9, other} ; {5, 7, other}]
map_label_card_3_1 = {0: 0, 1: 0, 2: 1, 3: 0, 4: 0, 5: 0, 6: 2, 7: 0, 8: 0, 9: 0}
map_label_card_3_2 = {0: 0, 1: 0, 2: 0, 3: 1, 4: 0, 5: 0, 6: 2, 7: 0, 8: 0, 9: 0}
map_label_card_3_3 = {0: 1, 1: 0, 2: 0, 3: 0, 4: 2, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0}
map_label_card_3_4 = {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 1, 9: 2}
map_label_card_3_5 = {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 1, 6: 0, 7: 2, 8: 0, 9: 0}

##### 2.1 XP1 - labels set $\mathcal{Card} = 3$ - $\mathcal{L}abels=\{2, 6, other\}$

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# ________________________    Data Loader    ________________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
## Load train data
batch_size = 512
train_loader, test_loader = load_and_process_data(bs=batch_size,
                                                  mapping_label_to_encode=map_label_card_3_1)


# -------------------------------------------------------------------
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
n_classes = 3
n_epochs = 20
lr = 0.005
dict_optim_cfg = {"momentum":0.9 ,"nesterov": True}
# -------------------------------------------------------------------
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
if run_toy_model:
    model_toy, optimizer_toy, criterion_toy = models_factory(model_type="toy_model",
                                                            optim_cfg=dict_optim_cfg,
                                                            nb_classes=n_classes)

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 instance    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# Initialize model, loss, optimizer and sgld sampler
## taking an untrained resnet18
if run_resnet:
    models_resnet, optimizer_resnet, criterion_resnet = models_factory(model_type="toy_model",
                                                                    optim_cfg=dict_optim_cfg,
                                                                    nb_classes=n_classes)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 training    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
if run_resnet:
    train_losses_resnet_xp1_6 = []
    val_losses_resnet_xp1_6 = []
    train_acc_resnet_xp1_6 = []
    val_acc_resnet_xp1_6 = []
    models_resnet_xp1_6 = []
    for epoch in range(n_epochs):
        train_loss, train_acc = train_step(model_resnet18,
                                        optimizer_resnet18,
                                        criterion_resnet18,
                                        train_loader)
        val_loss, val_acc = val_step(model_resnet18,
                                    criterion_resnet18,
                                    test_loader)
        train_losses_resnet_xp1_6.append(train_loss)
        val_losses_resnet_xp1_6.append(val_loss)
        train_acc_resnet_xp1_6.append(train_acc)
        val_acc_resnet_xp1_6.append(val_acc)
        models_resnet_xp1_6 += [copy.deepcopy(model_resnet18)]

        print_step_report(epoch=epoch,
                        train_loss=train_loss,
                        val_loss=val_loss,
                        train_acc=train_acc,
                        val_acc=val_acc)

    torch.save(models_resnet_xp1_6, "XP1/card_3-case_1/resnet_models.pth")
    export_loss_acc_to_csv(path="XP1/card_3-case_1/resnet_loss_acc.csv",
                            train_loss=train_losses_resnet_xp1_6,
                            test_loss=val_losses_resnet_xp1_6,
                            train_acc=train_acc_resnet_xp1_6,
                            test_acc=val_acc_resnet_xp1_6)

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model training    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
if run_toy_model:
    train_losses_toy_xp1_6 = []
    val_losses_toy_xp1_6 = []
    train_acc_toy_xp1_6 = []
    val_acc_toy_xp1_6 = []
    models_toy_xp1_6 = []
    for epoch in range(n_epochs):
        train_loss, train_acc = train_step(model_toy,
                                        optimizer_toy,
                                        criterion_toy,
                                        train_loader)
        val_loss, val_acc = val_step(model_toy,
                                    criterion_toy,
                                    test_loader)
        train_losses_toy_xp1_6.append(train_loss)
        val_losses_toy_xp1_6.append(val_loss)
        train_acc_toy_xp1_6.append(train_acc)
        val_acc_toy_xp1_6.append(val_acc)
        models_toy_xp1_6 += [copy.deepcopy(model_toy)]

        print_step_report(epoch=epoch,
                        train_loss=train_loss,
                        val_loss=val_loss,
                        train_acc=train_acc,
                        val_acc=val_acc)

    torch.save(models_toy_xp1_6, "XP1/card_3-case_1/toy_models.pth")
    export_loss_acc_to_csv(path="XP1/card_3-case_1/toy_loss_acc.csv",
                                train_loss=train_losses_toy_xp1_6,
                                test_loss=val_losses_toy_xp1_6,
                                train_acc=train_acc_toy_xp1_6,
                                test_acc=val_acc_toy_xp1_6)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy Model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

rlct_estimates_sgnht = []
rlct_estimates_sgld = []
for model in models_toy_xp1_6:
    rlct_estimate_sgnht = estimate_learning_coeff(
        model,
        train_loader,
        criterion=criterion_toy,
        optimizer_kwargs=dict(
            lr=1e-7,
            diffusion_factor=0.01,
            num_samples=len(train_data),
        ),
        sampling_method=SGNHT,
        num_chains=1,
        num_draws=400,
        num_burnin_steps=0,
        num_steps_bw_draws=1,
        device=DEVICE,
    )
    rlct_estimate_sgld = estimate_learning_coeff(
        model,
        train_loader,
        criterion=criterion_toy,
        optimizer_kwargs=dict(
            lr=1e-5,
            noise_level=1.0,
            elasticity=100.0,
            num_samples=len(train_data),
            temperature="adaptive",
        ),
        sampling_method=SGLD,
        num_chains=1,
        num_draws=400,
        num_burnin_steps=0,
        num_steps_bw_draws=1,
        device=DEVICE,
    )
    rlct_estimates_sgnht += [rlct_estimate_sgnht]
    rlct_estimates_sgld += [rlct_estimate_sgld]
    print(rlct_estimate_sgld, rlct_estimate_sgnht)

# plotting
export_rlct_to_csv("XP1/card_3-case_1/toy_rlct_estimated.csv",
                    rlct_estimates_sgnht=rlct_estimates_sgnht,
                    rlct_estimates_sgld=rlct_estimates_sgld)
plot_lambda_vs_epochs(train_losses_toy_xp1_6, val_losses_toy_xp1_6, rlct_estimates_sgnht, rlct_estimates_sgld)


##### 2.2 XP1 - labels set $\mathcal{Card} = 3$ - $\mathcal{L}abels=\{3, 6, other\}$

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# ________________________    Data Loader    ________________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
## Load train data
batch_size = 512
train_loader, test_loader = load_and_process_data(bs=batch_size,
                                                  mapping_label_to_encode=map_label_card_3_2)


# -------------------------------------------------------------------
# DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
n_classes = 3
n_epochs = 20
lr = 0.005
dict_optim_cfg = {"momentum":0.9 ,"nesterov": True}
# -------------------------------------------------------------------
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
if run_toy_model:
    model_toy, optimizer_toy, criterion_toy = models_factory(model_type="toy_model",
                                                            optim_cfg=dict_optim_cfg,
                                                            nb_classes=n_classes)

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 instance    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# Initialize model, loss, optimizer and sgld sampler
## taking an untrained resnet18
if run_resnet:
    models_resnet, optimizer_resnet, criterion_resnet = models_factory(model_type="toy_model",
                                                                    optim_cfg=dict_optim_cfg,
                                                                    nb_classes=n_classes)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 training    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

if run_resnet:
    train_losses_resnet_xp1_7 = []
    val_losses_resnet_xp1_7 = []
    train_acc_resnet_xp1_7 = []
    val_acc_resnet_xp1_7 = []
    models_resnet_xp1_7 = []
    for epoch in range(n_epochs):
        train_loss, train_acc = train_step(model_resnet18,
                                        optimizer_resnet18,
                                        criterion_resnet18,
                                        train_loader)
        val_loss, val_acc = val_step(model_resnet18,
                                    criterion_resnet18,
                                    test_loader)
        train_losses_resnet_xp1_7.append(train_loss)
        val_losses_resnet_xp1_7.append(val_loss)
        train_acc_resnet_xp1_7.append(train_acc)
        val_acc_resnet_xp1_7.append(val_acc)
        models_resnet_xp1_7 += [copy.deepcopy(model_resnet18)]

        print_step_report(epoch=epoch,
                        train_loss=train_loss,
                        val_loss=val_loss,
                        train_acc=train_acc,
                        val_acc=val_acc)

    torch.save(models_resnet_xp1_7, "XP1/card_3-case_2/resnet_models.pth")
    export_loss_acc_to_csv(path="XP1/card_3-case_2/resnet_loss_acc.csv",
                                train_loss=train_losses_resnet_xp1_7,
                                test_loss=val_losses_resnet_xp1_7,
                                train_acc=train_acc_resnet_xp1_7,
                                test_acc=val_acc_resnet_xp1_7)

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model training    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
if run_toy_model:
    train_losses_toy_xp1_7 = []
    val_losses_toy_xp1_7 = []
    train_acc_toy_xp1_7 = []
    val_acc_toy_xp1_7 = []
    models_toy_xp1_7 = []
    for epoch in range(n_epochs):
        train_loss, train_acc = train_step(model_toy,
                                        optimizer_toy,
                                        criterion_toy,
                                        train_loader)
        val_loss, val_acc = val_step(model_toy,
                                    criterion_toy,
                                    test_loader)
        train_losses_toy_xp1_7.append(train_loss)
        val_losses_toy_xp1_7.append(val_loss)
        train_acc_toy_xp1_7.append(train_acc)
        val_acc_toy_xp1_7.append(val_acc)
        models_toy_xp1_7 += [copy.deepcopy(model_toy)]

        print_step_report(epoch=epoch,
                        train_loss=train_loss,
                        val_loss=val_loss,
                        train_acc=train_acc,
                        val_acc=val_acc)

    torch.save(models_toy_xp1_7, "XP1/card_3-case_2/toy_models.pth")
    export_loss_acc_to_csv(path="XP1/card_3-case_2/toy_loss_acc.csv",
                                train_loss=train_losses_toy_xp1_7,
                                test_loss=val_losses_toy_xp1_7,
                                train_acc=train_acc_toy_xp1_7,
                                test_acc=val_acc_toy_xp1_7)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy Model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

rlct_estimates_sgnht = []
rlct_estimates_sgld = []
for model in models_toy_xp1_7:
    rlct_estimate_sgnht = estimate_learning_coeff(
        model,
        train_loader,
        criterion=criterion_toy,
        optimizer_kwargs=dict(
            lr=1e-7,
            diffusion_factor=0.01,
            num_samples=len(train_data),
        ),
        sampling_method=SGNHT,
        num_chains=1,
        num_draws=400,
        num_burnin_steps=0,
        num_steps_bw_draws=1,
        device=DEVICE,
    )
    rlct_estimate_sgld = estimate_learning_coeff(
        model,
        train_loader,
        criterion=criterion_toy,
        optimizer_kwargs=dict(
            lr=1e-5,
            noise_level=1.0,
            elasticity=100.0,
            num_samples=len(train_data),
            temperature="adaptive",
        ),
        sampling_method=SGLD,
        num_chains=1,
        num_draws=400,
        num_burnin_steps=0,
        num_steps_bw_draws=1,
        device=DEVICE,
    )
    rlct_estimates_sgnht += [rlct_estimate_sgnht]
    rlct_estimates_sgld += [rlct_estimate_sgld]
    print(rlct_estimate_sgld, rlct_estimate_sgnht)

# plotting
export_rlct_to_csv("XP1/card_3-case_2/toy_rlct_estimated.csv",
                    rlct_estimates_sgnht=rlct_estimates_sgnht,
                    rlct_estimates_sgld=rlct_estimates_sgld)
plot_lambda_vs_epochs(train_losses_toy_xp1_7, val_losses_toy_xp1_7, rlct_estimates_sgnht, rlct_estimates_sgld)


##### 2.3 XP1 - labels set $\mathcal{Card} = 3$ - $\mathcal{L}abels=\{0, 4, other\}$

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# ________________________    Data Loader    ________________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
## Load train data
batch_size = 512
train_loader, test_loader = load_and_process_data(bs=batch_size,
                                                  mapping_label_to_encode=map_label_card_3_3)


# -------------------------------------------------------------------
# DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
n_classes = 3
n_epochs = 20
lr = 0.005
dict_optim_cfg = {"momentum":0.9 ,"nesterov": True}
# -------------------------------------------------------------------
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
if run_toy_model:
    model_toy, optimizer_toy, criterion_toy = models_factory(model_type="toy_model",
                                                            optim_cfg=dict_optim_cfg,
                                                            nb_classes=n_classes)

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 instance    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# Initialize model, loss, optimizer and sgld sampler
## taking an untrained resnet18
if run_resnet:
    models_resnet, optimizer_resnet, criterion_resnet = models_factory(model_type="toy_model",
                                                                    optim_cfg=dict_optim_cfg,
                                                                    nb_classes=n_classes)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 training    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
if run_resnet:
    train_losses_resnet_xp1_8 = []
    val_losses_resnet_xp1_8 = []
    train_acc_resnet_xp1_8 = []
    val_acc_resnet_xp1_8 = []
    models_resnet_xp1_8 = []
    for epoch in range(n_epochs):
        train_loss, train_acc = train_step(model_resnet18,
                                        optimizer_resnet18,
                                        criterion_resnet18,
                                        train_loader)
        val_loss, val_acc = val_step(model_resnet18,
                                    criterion_resnet18,
                                    test_loader)
        train_losses_resnet_xp1_8.append(train_loss)
        val_losses_resnet_xp1_8.append(val_loss)
        train_acc_resnet_xp1_8.append(train_acc)
        val_acc_resnet_xp1_8.append(val_acc)
        models_resnet_xp1_8 += [copy.deepcopy(model_resnet18)]

        print_step_report(epoch=epoch,
                        train_loss=train_loss,
                        val_loss=val_loss,
                        train_acc=train_acc,
                        val_acc=val_acc)

    torch.save(models_resnet_xp1_8, "XP1/card_3-case_3/resnet_models.pth")
    export_loss_acc_to_csv(path="XP1/card_3-case_3/resnet_loss_acc.csv",
                                train_loss=train_losses_resnet_xp1_8,
                                test_loss=val_losses_resnet_xp1_8,
                                train_acc=train_acc_resnet_xp1_8,
                                test_acc=val_acc_resnet_xp1_8)

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model training    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
if run_toy_model:
    train_losses_toy_xp1_8 = []
    val_losses_toy_xp1_8 = []
    train_acc_toy_xp1_8 = []
    val_acc_toy_xp1_8 = []
    models_toy_xp1_8 = []
    for epoch in range(n_epochs):
        train_loss, train_acc = train_step(model_toy,
                                        optimizer_toy,
                                        criterion_toy,
                                        train_loader)
        val_loss, val_acc = val_step(model_toy,
                                    criterion_toy,
                                    test_loader)
        train_losses_toy_xp1_8.append(train_loss)
        val_losses_toy_xp1_8.append(val_loss)
        train_acc_toy_xp1_8.append(train_acc)
        val_acc_toy_xp1_8.append(val_acc)
        models_toy_xp1_8 += [copy.deepcopy(model_toy)]

        print_step_report(epoch=epoch,
                        train_loss=train_loss,
                        val_loss=val_loss,
                        train_acc=train_acc,
                        val_acc=val_acc)

    torch.save(models_toy_xp1_8, "XP1/card_3-case_3/toy_models.pth")
    export_loss_acc_to_csv(path="XP1/card_3-case_3/toy_loss_acc.csv",
                                train_loss=train_losses_toy_xp1_8,
                                test_loss=val_losses_toy_xp1_8,
                                train_acc=train_acc_toy_xp1_8,
                                test_acc=val_acc_toy_xp1_8)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy Model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

rlct_estimates_sgnht = []
rlct_estimates_sgld = []
for model in models_toy_xp1_8:
    rlct_estimate_sgnht = estimate_learning_coeff(
        model,
        train_loader,
        criterion=criterion_toy,
        optimizer_kwargs=dict(
            lr=1e-7,
            diffusion_factor=0.01,
            num_samples=len(train_data),
        ),
        sampling_method=SGNHT,
        num_chains=1,
        num_draws=400,
        num_burnin_steps=0,
        num_steps_bw_draws=1,
        device=DEVICE,
    )
    rlct_estimate_sgld = estimate_learning_coeff(
        model,
        train_loader,
        criterion=criterion_toy,
        optimizer_kwargs=dict(
            lr=1e-5,
            noise_level=1.0,
            elasticity=100.0,
            num_samples=len(train_data),
            temperature="adaptive",
        ),
        sampling_method=SGLD,
        num_chains=1,
        num_draws=400,
        num_burnin_steps=0,
        num_steps_bw_draws=1,
        device=DEVICE,
    )
    rlct_estimates_sgnht += [rlct_estimate_sgnht]
    rlct_estimates_sgld += [rlct_estimate_sgld]
    print(rlct_estimate_sgld, rlct_estimate_sgnht)

# plotting
export_rlct_to_csv("XP1/card_3-case_3/toy_rlct_estimated.csv",
                    rlct_estimates_sgnht=rlct_estimates_sgnht,
                    rlct_estimates_sgld=rlct_estimates_sgld)
plot_lambda_vs_epochs(train_losses_toy_xp1_8, val_losses_toy_xp1_8, rlct_estimates_sgnht, rlct_estimates_sgld)


##### 2.4 XP1 - labels set $\mathcal{Card} = 3$ - $\mathcal{L}abels=\{8, 9, other\}$

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# ________________________    Data Loader    ________________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
## Load train data
batch_size = 512
train_loader, test_loader = load_and_process_data(bs=batch_size,
                                                  mapping_label_to_encode=map_label_card_3_4)


# -------------------------------------------------------------------
# DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
n_classes = 3
n_epochs = 20
lr = 0.005
dict_optim_cfg = {"momentum":0.9 ,"nesterov": True}
# -------------------------------------------------------------------
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
if run_toy_model:
    model_toy, optimizer_toy, criterion_toy = models_factory(model_type="toy_model",
                                                            optim_cfg=dict_optim_cfg,
                                                            nb_classes=n_classes)

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 instance    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# Initialize model, loss, optimizer and sgld sampler
## taking an untrained resnet18
if run_resnet:
    models_resnet, optimizer_resnet, criterion_resnet = models_factory(model_type="toy_model",
                                                                    optim_cfg=dict_optim_cfg,
                                                                    nb_classes=n_classes)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 training    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
if run_resnet:
    train_losses_resnet_xp1_9 = []
    val_losses_resnet_xp1_9 = []
    train_acc_resnet_xp1_9 = []
    val_acc_resnet_xp1_9 = []
    models_resnet_xp1_9 = []
    for epoch in range(n_epochs):
        train_loss, train_acc = train_step(model_resnet18,
                                        optimizer_resnet18,
                                        criterion_resnet18,
                                        train_loader)
        val_loss, val_acc = val_step(model_resnet18,
                                    criterion_resnet18,
                                    test_loader)
        train_losses_resnet_xp1_9.append(train_loss)
        val_losses_resnet_xp1_9.append(val_loss)
        train_acc_resnet_xp1_9.append(train_acc)
        val_acc_resnet_xp1_9.append(val_acc)
        models_resnet_xp1_9 += [copy.deepcopy(model_resnet18)]

        print_step_report(epoch=epoch,
                        train_loss=train_loss,
                        val_loss=val_loss,
                        train_acc=train_acc,
                        val_acc=val_acc)

    torch.save(models_resnet_xp1_9, "XP1/card_3-case_4/resnet_models.pth")
    export_loss_acc_to_csv(path="XP1/card_3-case_4/resnet_loss_acc.csv",
                                train_loss=train_losses_resnet_xp1_9,
                                test_loss=val_losses_resnet_xp1_9,
                                train_acc=train_acc_resnet_xp1_9,
                                test_acc=val_acc_resnet_xp1_9)

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model training    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
if run_toy_model:
    train_losses_toy_xp1_9 = []
    val_losses_toy_xp1_9 = []
    train_acc_toy_xp1_9 = []
    val_acc_toy_xp1_9 = []
    models_toy_xp1_9 = []
    for epoch in range(n_epochs):
        train_loss, train_acc = train_step(model_toy,
                                        optimizer_toy,
                                        criterion_toy,
                                        train_loader)
        val_loss, val_acc = val_step(model_toy,
                                    criterion_toy,
                                    test_loader)
        train_losses_toy_xp1_9.append(train_loss)
        val_losses_toy_xp1_9.append(val_loss)
        train_acc_toy_xp1_9.append(train_acc)
        val_acc_toy_xp1_9.append(val_acc)
        models_toy_xp1_9 += [copy.deepcopy(model_toy)]

        print_step_report(epoch=epoch,
                        train_loss=train_loss,
                        val_loss=val_loss,
                        train_acc=train_acc,
                        val_acc=val_acc)

    torch.save(models_toy_xp1_9, "XP1/card_3-case_4/toy_models.pth")
    export_loss_acc_to_csv(path="XP1/card_3-case_4/toy_loss_acc.csv",
                                train_loss=train_losses_toy_xp1_9,
                                test_loss=val_losses_toy_xp1_9,
                                train_acc=train_acc_toy_xp1_9,
                                test_acc=val_acc_toy_xp1_9)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy Model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

rlct_estimates_sgnht = []
rlct_estimates_sgld = []
for model in models_toy_xp1_9:
    rlct_estimate_sgnht = estimate_learning_coeff(
        model,
        train_loader,
        criterion=criterion_toy,
        optimizer_kwargs=dict(
            lr=1e-7,
            diffusion_factor=0.01,
            num_samples=len(train_data),
        ),
        sampling_method=SGNHT,
        num_chains=1,
        num_draws=400,
        num_burnin_steps=0,
        num_steps_bw_draws=1,
        device=DEVICE,
    )
    rlct_estimate_sgld = estimate_learning_coeff(
        model,
        train_loader,
        criterion=criterion_toy,
        optimizer_kwargs=dict(
            lr=1e-5,
            noise_level=1.0,
            elasticity=100.0,
            num_samples=len(train_data),
            temperature="adaptive",
        ),
        sampling_method=SGLD,
        num_chains=1,
        num_draws=400,
        num_burnin_steps=0,
        num_steps_bw_draws=1,
        device=DEVICE,
    )
    rlct_estimates_sgnht += [rlct_estimate_sgnht]
    rlct_estimates_sgld += [rlct_estimate_sgld]
    print(rlct_estimate_sgld, rlct_estimate_sgnht)

# plotting
export_rlct_to_csv("XP1/card_3-case_4/toy_rlct_estimated.csv",
                    rlct_estimates_sgnht=rlct_estimates_sgnht,
                    rlct_estimates_sgld=rlct_estimates_sgld)
plot_lambda_vs_epochs(train_losses_toy_xp1_9, val_losses_toy_xp1_9, rlct_estimates_sgnht, rlct_estimates_sgld)


##### 2.5 XP1 - labels set $\mathcal{Card} = 3$ - $\mathcal{L}abels=\{5, 7, other\}$

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# ________________________    Data Loader    ________________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
## Load train data
batch_size = 512
train_loader, test_loader = load_and_process_data(bs=batch_size,
                                                  mapping_label_to_encode=map_label_card_3_5)


# -------------------------------------------------------------------
# DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
n_classes = 3
n_epochs = 20
lr = 0.005
dict_optim_cfg = {"momentum":0.9 ,"nesterov": True}
# -------------------------------------------------------------------
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
if run_toy_model:
    model_toy, optimizer_toy, criterion_toy = models_factory(model_type="toy_model",
                                                            optim_cfg=dict_optim_cfg,
                                                            nb_classes=n_classes)

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 instance    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# Initialize model, loss, optimizer and sgld sampler
## taking an untrained resnet18
if run_resnet:
    models_resnet, optimizer_resnet, criterion_resnet = models_factory(model_type="toy_model",
                                                                    optim_cfg=dict_optim_cfg,
                                                                    nb_classes=n_classes)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 training    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
if run_resnet:
    train_losses_resnet_xp1_10 = []
    val_losses_resnet_xp1_10 = []
    models_resnet_xp1_10 = []
    for epoch in range(n_epochs):
        train_loss, train_acc = train_step(model_resnet18,
                                        optimizer_resnet18,
                                        criterion_resnet18,
                                        train_loader)
        val_loss, val_acc = val_step(model_resnet18,
                                    criterion_resnet18,
                                    test_loader)
        train_losses_resnet_xp1_10.append(train_loss)
        val_losses_resnet_xp1_10.append(val_loss)
        models_resnet_xp1_10 += [copy.deepcopy(model_resnet18)]

        print_step_report(epoch=epoch,
                        train_loss=train_loss,
                        val_loss=val_loss,
                        train_acc=train_acc,
                        val_acc=val_acc)

    torch.save(models_resnet_xp1_10, "XP1/card_3-case_5/resnet_models.pth")

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model training    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
if run_toy_model:
    train_losses_toy_xp1_10 = []
    val_losses_toy_xp1_10 = []
    train_acc_toy_xp1_10 = []
    val_acc_toy_xp1_10 = []
    models_toy_xp1_10 = []
    for epoch in range(n_epochs):
        train_loss, train_acc = train_step(model_toy,
                                        optimizer_toy,
                                        criterion_toy,
                                        train_loader)
        val_loss, val_acc = val_step(model_toy,
                                    criterion_toy,
                                    test_loader)
        train_losses_toy_xp1_10.append(train_loss)
        val_losses_toy_xp1_10.append(val_loss)
        train_acc_toy_xp1_10.append(train_acc)
        val_acc_toy_xp1_10.append(val_acc)
        models_toy_xp1_10 += [copy.deepcopy(model_toy)]

        print_step_report(epoch=epoch,
                        train_loss=train_loss,
                        val_loss=val_loss,
                        train_acc=train_acc,
                        val_acc=val_acc)

    torch.save(models_toy_xp1_10, "XP1/card_3-case_5/toy_models.pth")
    export_loss_acc_to_csv(path="XP1/card_3-case_5/toy_loss_acc.csv",
                                train_loss=train_losses_toy_xp1_10,
                                test_loss=val_losses_toy_xp1_10,
                                train_acc=train_acc_toy_xp1_10,
                                test_acc=val_acc_toy_xp1_10)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy Model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

rlct_estimates_sgnht = []
rlct_estimates_sgld = []
for model in models_toy_xp1_10:
    rlct_estimate_sgnht = estimate_learning_coeff(
        model,
        train_loader,
        criterion=criterion_toy,
        optimizer_kwargs=dict(
            lr=1e-7,
            diffusion_factor=0.01,
            num_samples=len(train_data),
        ),
        sampling_method=SGNHT,
        num_chains=1,
        num_draws=400,
        num_burnin_steps=0,
        num_steps_bw_draws=1,
        device=DEVICE,
    )
    rlct_estimate_sgld = estimate_learning_coeff(
        model,
        train_loader,
        criterion=criterion_toy,
        optimizer_kwargs=dict(
            lr=1e-5,
            noise_level=1.0,
            elasticity=100.0,
            num_samples=len(train_data),
            temperature="adaptive",
        ),
        sampling_method=SGLD,
        num_chains=1,
        num_draws=400,
        num_burnin_steps=0,
        num_steps_bw_draws=1,
        device=DEVICE,
    )
    rlct_estimates_sgnht += [rlct_estimate_sgnht]
    rlct_estimates_sgld += [rlct_estimate_sgld]
    print(rlct_estimate_sgld, rlct_estimate_sgnht)

# plotting
export_rlct_to_csv("XP1/card_3-case_5/toy_rlct_estimated.csv",
                    rlct_estimates_sgnht=rlct_estimates_sgnht,
                    rlct_estimates_sgld=rlct_estimates_sgld)
plot_lambda_vs_epochs(train_losses_toy_xp1_10, val_losses_toy_xp1_10, rlct_estimates_sgnht, rlct_estimates_sgld)


### Settings and runs label sets $Card(\mathcal{L}abels)=4$

* Labels modification and data loader
* Toy model for 4 classes
* Resnet18 model for 4 classes

In [ ]:
# keys are the original label, values are the new labels
# [{0, 5, 6, other} ; {6, 7, 9, other} ; {2, 3, 5, other} ; {0, 1, 4, other}]
map_label_card_4_1 = {0: 1, 1: 0, 2: 0, 3: 0, 4: 0, 5: 2, 6: 3, 7: 0, 8: 0, 9: 0}
map_label_card_4_2 = {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 1, 7: 2, 8: 0, 9: 3}
map_label_card_4_3 = {0: 0, 1: 0, 2: 1, 3: 2, 4: 0, 5: 3, 6: 0, 7: 0, 8: 0, 9: 0}
map_label_card_4_4 = {0: 1, 1: 2, 2: 0, 3: 0, 4: 3, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0}

##### 3.1 XP1 - labels set $\mathcal{Card} = 4$ - $\mathcal{L}abels=\{0, 5, 6, other\}$

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# ________________________    Data Loader    ________________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
## Load train data
batch_size = 512
train_loader, test_loader = load_and_process_data(bs=batch_size,
                                                  mapping_label_to_encode=map_label_card_4_1)


# -------------------------------------------------------------------
# DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
n_classes = 4
n_epochs = 20
lr = 0.005
dict_optim_cfg = {"momentum":0.9 ,"nesterov": True}
# -------------------------------------------------------------------
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
if run_toy_model:
    model_toy, optimizer_toy, criterion_toy = models_factory(model_type="toy_model",
                                                            optim_cfg=dict_optim_cfg,
                                                            nb_classes=n_classes)

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 instance    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# Initialize model, loss, optimizer and sgld sampler
## taking an untrained resnet18
if run_resnet:
    models_resnet, optimizer_resnet, criterion_resnet = models_factory(model_type="toy_model",
                                                                    optim_cfg=dict_optim_cfg,
                                                                    nb_classes=n_classes)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 training    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
if run_resnet:
    train_losses_resnet_xp1_11 = []
    val_losses_resnet_xp1_11 = []
    models_resnet_xp1_11 = []
    for epoch in range(n_epochs):
        train_loss, train_acc = train_step(model_resnet18,
                                        optimizer_resnet18,
                                        criterion_resnet18,
                                        train_loader)
        val_loss, val_acc = val_step(model_resnet18,
                                    criterion_resnet18,
                                    test_loader)
        train_losses_resnet_xp1_11.append(train_loss)
        val_losses_resnet_xp1_11.append(val_loss)
        models_resnet_xp1_11 += [copy.deepcopy(model_resnet18)]

        print_step_report(epoch=epoch,
                        train_loss=train_loss,
                        val_loss=val_loss,
                        train_acc=train_acc,
                        val_acc=val_acc)

    torch.save(models_resnet_xp1_11, "XP1/card_4-case_1/resnet_models.pth")

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model training    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
if run_toy_model:
    train_losses_toy_xp1_11 = []
    val_losses_toy_xp1_11 = []
    train_acc_toy_xp1_11 = []
    val_acc_toy_xp1_11 = []
    models_toy_xp1_11 = []
    for epoch in range(n_epochs):
        train_loss, train_acc = train_step(model_toy,
                                        optimizer_toy,
                                        criterion_toy,
                                        train_loader)
        val_loss, val_acc = val_step(model_toy,
                                    criterion_toy,
                                    test_loader)
        train_losses_toy_xp1_11.append(train_loss)
        val_losses_toy_xp1_11.append(val_loss)
        train_acc_toy_xp1_11.append(train_acc)
        val_acc_toy_xp1_11.append(val_acc)
        models_toy_xp1_11 += [copy.deepcopy(model_toy)]

        print_step_report(epoch=epoch,
                        train_loss=train_loss,
                        val_loss=val_loss,
                        train_acc=train_acc,
                        val_acc=val_acc)

    torch.save(models_toy_xp1_11, "XP1/card_4-case_1/toy_models.pth")
    export_loss_acc_to_csv(path="XP1/card_4-case_1/toy_loss_acc.csv",
                                train_loss=train_losses_toy_xp1_11,
                                test_loss=val_losses_toy_xp1_11,
                                train_acc=train_acc_toy_xp1_11,
                                test_acc=val_acc_toy_xp1_11)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy Model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

rlct_estimates_sgnht = []
rlct_estimates_sgld = []
for model in models_toy_xp1_11:
    rlct_estimate_sgnht = estimate_learning_coeff(
        model,
        train_loader,
        criterion=criterion_toy,
        optimizer_kwargs=dict(
            lr=1e-7,
            diffusion_factor=0.01,
            num_samples=len(train_data),
        ),
        sampling_method=SGNHT,
        num_chains=1,
        num_draws=400,
        num_burnin_steps=0,
        num_steps_bw_draws=1,
        device=DEVICE,
    )
    rlct_estimate_sgld = estimate_learning_coeff(
        model,
        train_loader,
        criterion=criterion_toy,
        optimizer_kwargs=dict(
            lr=1e-5,
            noise_level=1.0,
            elasticity=100.0,
            num_samples=len(train_data),
            temperature="adaptive",
        ),
        sampling_method=SGLD,
        num_chains=1,
        num_draws=400,
        num_burnin_steps=0,
        num_steps_bw_draws=1,
        device=DEVICE,
    )
    rlct_estimates_sgnht += [rlct_estimate_sgnht]
    rlct_estimates_sgld += [rlct_estimate_sgld]
    print(rlct_estimate_sgld, rlct_estimate_sgnht)

# plotting
export_rlct_to_csv("XP1/card_4-case_1/toy_rlct_estimated.csv",
                    rlct_estimates_sgnht=rlct_estimates_sgnht,
                    rlct_estimates_sgld=rlct_estimates_sgld)
plot_lambda_vs_epochs(train_losses_toy_xp1_11, val_losses_toy_xp1_11, rlct_estimates_sgnht, rlct_estimates_sgld)


##### 3.2 XP1 - labels set $\mathcal{Card} = 4$ - $\mathcal{L}abels=\{6, 7, 9, other\}$

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# ________________________    Data Loader    ________________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
## Load train data
batch_size = 512
train_loader, test_loader = load_and_process_data(bs=batch_size,
                                                  mapping_label_to_encode=map_label_card_4_2)


# -------------------------------------------------------------------
# DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
n_classes = 4
n_epochs = 20
lr = 0.005
dict_optim_cfg = {"momentum":0.9 ,"nesterov": True}
# -------------------------------------------------------------------
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
if run_toy_model:
    model_toy, optimizer_toy, criterion_toy = models_factory(model_type="toy_model",
                                                            optim_cfg=dict_optim_cfg,
                                                            nb_classes=n_classes)

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 instance    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# Initialize model, loss, optimizer and sgld sampler
## taking an untrained resnet18
if run_resnet:
    models_resnet, optimizer_resnet, criterion_resnet = models_factory(model_type="toy_model",
                                                                    optim_cfg=dict_optim_cfg,
                                                                    nb_classes=n_classes)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 training    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
if run_resnet:
    train_losses_resnet_xp1_12 = []
    val_losses_resnet_xp1_12 = []
    models_resnet_xp1_12 = []
    for epoch in range(n_epochs):
        train_loss, train_acc = train_step(model_resnet18,
                                        optimizer_resnet18,
                                        criterion_resnet18,
                                        train_loader)
        val_loss, val_acc = val_step(model_resnet18,
                                    criterion_resnet18,
                                    test_loader)
        train_losses_resnet_xp1_12.append(train_loss)
        val_losses_resnet_xp1_12.append(val_loss)
        models_resnet_xp1_12 += [copy.deepcopy(model_resnet18)]

        print_step_report(epoch=epoch,
                        train_loss=train_loss,
                        val_loss=val_loss,
                        train_acc=train_acc,
                        val_acc=val_acc)

    torch.save(models_resnet_xp1_12, "XP1/card_4-case_2/resnet_models.pth")

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model training    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
if run_toy_model:
    train_losses_toy_xp1_12 = []
    val_losses_toy_xp1_12 = []
    train_acc_toy_xp1_12 = []
    val_acc_toy_xp1_12 = []
    models_toy_xp1_12 = []
    for epoch in range(n_epochs):
        train_loss, train_acc = train_step(model_toy,
                                        optimizer_toy,
                                        criterion_toy,
                                        train_loader)
        val_loss, val_acc = val_step(model_toy,
                                    criterion_toy,
                                    test_loader)
        train_losses_toy_xp1_12.append(train_loss)
        val_losses_toy_xp1_12.append(val_loss)
        train_acc_toy_xp1_12.append(train_acc)
        val_acc_toy_xp1_12.append(val_acc)
        models_toy_xp1_12 += [copy.deepcopy(model_toy)]

        print_step_report(epoch=epoch,
                        train_loss=train_loss,
                        val_loss=val_loss,
                        train_acc=train_acc,
                        val_acc=val_acc)

    torch.save(models_toy_xp1_12, "XP1/card_4-case_2/toy_models.pth")
    export_loss_acc_to_csv(path="XP1/card_4-case_2/toy_loss_acc.csv",
                                train_loss=train_losses_toy_xp1_12,
                                test_loss=val_losses_toy_xp1_12,
                                train_acc=train_acc_toy_xp1_12,
                                test_acc=val_acc_toy_xp1_12)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy Model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

rlct_estimates_sgnht = []
rlct_estimates_sgld = []
for model in models_toy_xp1_12:
    rlct_estimate_sgnht = estimate_learning_coeff(
        model,
        train_loader,
        criterion=criterion_toy,
        optimizer_kwargs=dict(
            lr=1e-7,
            diffusion_factor=0.01,
            num_samples=len(train_data),
        ),
        sampling_method=SGNHT,
        num_chains=1,
        num_draws=400,
        num_burnin_steps=0,
        num_steps_bw_draws=1,
        device=DEVICE,
    )
    rlct_estimate_sgld = estimate_learning_coeff(
        model,
        train_loader,
        criterion=criterion_toy,
        optimizer_kwargs=dict(
            lr=1e-5,
            noise_level=1.0,
            elasticity=100.0,
            num_samples=len(train_data),
            temperature="adaptive",
        ),
        sampling_method=SGLD,
        num_chains=1,
        num_draws=400,
        num_burnin_steps=0,
        num_steps_bw_draws=1,
        device=DEVICE,
    )
    rlct_estimates_sgnht += [rlct_estimate_sgnht]
    rlct_estimates_sgld += [rlct_estimate_sgld]
    print(rlct_estimate_sgld, rlct_estimate_sgnht)

# plotting
export_rlct_to_csv("XP1/card_4-case_2/toy_rlct_estimated.csv",
                    rlct_estimates_sgnht=rlct_estimates_sgnht,
                    rlct_estimates_sgld=rlct_estimates_sgld)
plot_lambda_vs_epochs(train_losses_toy_xp1_12, val_losses_toy_xp1_12, rlct_estimates_sgnht, rlct_estimates_sgld)


##### 3.3 XP1 - labels set $\mathcal{Card} = 4$ - $\mathcal{L}abels=\{2, 3, 5, other\}$

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# ________________________    Data Loader    ________________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
## Load train data
batch_size = 512
train_loader, test_loader = load_and_process_data(bs=batch_size,
                                                  mapping_label_to_encode=map_label_card_4_3)


# -------------------------------------------------------------------
# DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
n_classes = 4
n_epochs = 20
lr = 0.005
dict_optim_cfg = {"momentum":0.9 ,"nesterov": True}
# -------------------------------------------------------------------
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
if run_toy_model:
    model_toy, optimizer_toy, criterion_toy = models_factory(model_type="toy_model",
                                                            optim_cfg=dict_optim_cfg,
                                                            nb_classes=n_classes)

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 instance    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# Initialize model, loss, optimizer and sgld sampler
## taking an untrained resnet18
if run_resnet:
    models_resnet, optimizer_resnet, criterion_resnet = models_factory(model_type="toy_model",
                                                                    optim_cfg=dict_optim_cfg,
                                                                    nb_classes=n_classes)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 training    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
if run_resnet:
    train_losses_resnet_xp1_13 = []
    val_losses_resnet_xp1_13 = []
    models_resnet_xp1_13 = []
    for epoch in range(n_epochs):
        train_loss, train_acc = train_step(model_resnet18,
                                        optimizer_resnet18,
                                        criterion_resnet18,
                                        train_loader)
        val_loss, val_acc = val_step(model_resnet18,
                                    criterion_resnet18,
                                    test_loader)
        train_losses_resnet_xp1_13.append(train_loss)
        val_losses_resnet_xp1_13.append(val_loss)
        models_resnet_xp1_13 += [copy.deepcopy(model_resnet18)]

        print_step_report(epoch=epoch,
                        train_loss=train_loss,
                        val_loss=val_loss,
                        train_acc=train_acc,
                        val_acc=val_acc)

    torch.save(models_resnet_xp1_13, "XP1/card_4-case_3/resnet_models.pth")

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model training    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
if run_toy_model:
    train_losses_toy_xp1_13 = []
    val_losses_toy_xp1_13 = []
    train_acc_toy_xp1_13 = []
    val_acc_toy_xp1_13 = []
    models_toy_xp1_13 = []
    for epoch in range(n_epochs):
        train_loss, train_acc = train_step(model_toy,
                                        optimizer_toy,
                                        criterion_toy,
                                        train_loader)
        val_loss, val_acc = val_step(model_toy,
                                    criterion_toy,
                                    test_loader)
        train_losses_toy_xp1_13.append(train_loss)
        val_losses_toy_xp1_13.append(val_loss)
        train_acc_toy_xp1_13.append(train_acc)
        val_acc_toy_xp1_13.append(val_acc)
        models_toy_xp1_13 += [copy.deepcopy(model_toy)]

        print_step_report(epoch=epoch,
                        train_loss=train_loss,
                        val_loss=val_loss,
                        train_acc=train_acc,
                        val_acc=val_acc)

    torch.save(models_toy_xp1_13, "XP1/card_4-case_3/toy_models.pth")
    export_loss_acc_to_csv(path="XP1/card_4-case_3/toy_loss_acc.csv",
                                train_loss=train_losses_toy_xp1_13,
                                test_loss=val_losses_toy_xp1_13,
                                train_acc=train_acc_toy_xp1_13,
                                test_acc=val_acc_toy_xp1_13)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy Model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

rlct_estimates_sgnht = []
rlct_estimates_sgld = []
for model in models_toy_xp1_13:
    rlct_estimate_sgnht = estimate_learning_coeff(
        model,
        train_loader,
        criterion=criterion_toy,
        optimizer_kwargs=dict(
            lr=1e-7,
            diffusion_factor=0.01,
            num_samples=len(train_data),
        ),
        sampling_method=SGNHT,
        num_chains=1,
        num_draws=400,
        num_burnin_steps=0,
        num_steps_bw_draws=1,
        device=DEVICE,
    )
    rlct_estimate_sgld = estimate_learning_coeff(
        model,
        train_loader,
        criterion=criterion_toy,
        optimizer_kwargs=dict(
            lr=1e-5,
            noise_level=1.0,
            elasticity=100.0,
            num_samples=len(train_data),
            temperature="adaptive",
        ),
        sampling_method=SGLD,
        num_chains=1,
        num_draws=400,
        num_burnin_steps=0,
        num_steps_bw_draws=1,
        device=DEVICE,
    )
    rlct_estimates_sgnht += [rlct_estimate_sgnht]
    rlct_estimates_sgld += [rlct_estimate_sgld]
    print(rlct_estimate_sgld, rlct_estimate_sgnht)

# plotting
export_rlct_to_csv("XP1/card_4-case_3/toy_rlct_estimated.csv",
                    rlct_estimates_sgnht=rlct_estimates_sgnht,
                    rlct_estimates_sgld=rlct_estimates_sgld)
plot_lambda_vs_epochs(train_losses_toy_xp1_13, val_losses_toy_xp1_13, rlct_estimates_sgnht, rlct_estimates_sgld)


##### 3.4 XP1 - labels set $\mathcal{Card} = 4$ - $\mathcal{L}abels=\{0, 1, 4, other\}$

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# ________________________    Data Loader    ________________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
## Load train data
batch_size = 512
train_loader, test_loader = load_and_process_data(bs=batch_size,
                                                  mapping_label_to_encode=map_label_card_4_4)


# -------------------------------------------------------------------
# DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
n_classes = 4
n_epochs = 20
lr = 0.005
dict_optim_cfg = {"momentum":0.9 ,"nesterov": True}
# -------------------------------------------------------------------
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
if run_toy_model:
    model_toy, optimizer_toy, criterion_toy = models_factory(model_type="toy_model",
                                                            optim_cfg=dict_optim_cfg,
                                                            nb_classes=n_classes)

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 instance    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# Initialize model, loss, optimizer and sgld sampler
## taking an untrained resnet18
if run_resnet:
    models_resnet, optimizer_resnet, criterion_resnet = models_factory(model_type="toy_model",
                                                                    optim_cfg=dict_optim_cfg,
                                                                    nb_classes=n_classes)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 training    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
if run_resnet:
    train_losses_resnet_xp1_14 = []
    val_losses_resnet_xp1_14 = []
    models_resnet_xp1_14 = []
    for epoch in range(n_epochs):
        train_loss, train_acc = train_step(model_resnet18,
                                        optimizer_resnet18,
                                        criterion_resnet18,
                                        train_loader)
        val_loss, val_acc = val_step(model_resnet18,
                                    criterion_resnet18,
                                    test_loader)
        train_losses_resnet_xp1_14.append(train_loss)
        val_losses_resnet_xp1_14.append(val_loss)
        models_resnet_xp1_14 += [copy.deepcopy(model_resnet18)]

        print_step_report(epoch=epoch,
                        train_loss=train_loss,
                        val_loss=val_loss,
                        train_acc=train_acc,
                        val_acc=val_acc)

    torch.save(models_resnet_xp1_14, "XP1/card_4-case_4/resnet_models.pth")

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model training    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
if run_toy_model:
    train_losses_toy_xp1_14 = []
    val_losses_toy_xp1_14 = []
    train_acc_toy_xp1_14 = []
    val_acc_toy_xp1_14 = []
    models_toy_xp1_14 = []
    for epoch in range(n_epochs):
        train_loss, train_acc = train_step(model_toy,
                                        optimizer_toy,
                                        criterion_toy,
                                        train_loader)
        val_loss, val_acc = val_step(model_toy,
                                    criterion_toy,
                                    test_loader)
        train_losses_toy_xp1_14.append(train_loss)
        val_losses_toy_xp1_14.append(val_loss)
        train_acc_toy_xp1_14.append(train_acc)
        val_acc_toy_xp1_14.append(val_acc)
        models_toy_xp1_14 += [copy.deepcopy(model_toy)]

        print_step_report(epoch=epoch,
                        train_loss=train_loss,
                        val_loss=val_loss,
                        train_acc=train_acc,
                        val_acc=val_acc)

    torch.save(models_toy_xp1_14, "XP1/card_4-case_4/toy_models.pth")
    export_loss_acc_to_csv(path="XP1/card_4-case_4/toy_loss_acc.csv",
                                train_loss=train_losses_toy_xp1_14,
                                test_loss=val_losses_toy_xp1_14,
                                train_acc=train_acc_toy_xp1_14,
                                test_acc=val_acc_toy_xp1_14)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy Model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

rlct_estimates_sgnht = []
rlct_estimates_sgld = []
for model in models_toy_xp1_14:
    rlct_estimate_sgnht = estimate_learning_coeff(
        model,
        train_loader,
        criterion=criterion_toy,
        optimizer_kwargs=dict(
            lr=1e-7,
            diffusion_factor=0.01,
            num_samples=len(train_data),
        ),
        sampling_method=SGNHT,
        num_chains=1,
        num_draws=400,
        num_burnin_steps=0,
        num_steps_bw_draws=1,
        device=DEVICE,
    )
    rlct_estimate_sgld = estimate_learning_coeff(
        model,
        train_loader,
        criterion=criterion_toy,
        optimizer_kwargs=dict(
            lr=1e-5,
            noise_level=1.0,
            elasticity=100.0,
            num_samples=len(train_data),
            temperature="adaptive",
        ),
        sampling_method=SGLD,
        num_chains=1,
        num_draws=400,
        num_burnin_steps=0,
        num_steps_bw_draws=1,
        device=DEVICE,
    )
    rlct_estimates_sgnht += [rlct_estimate_sgnht]
    rlct_estimates_sgld += [rlct_estimate_sgld]
    print(rlct_estimate_sgld, rlct_estimate_sgnht)

# plotting
export_rlct_to_csv("XP1/card_4-case_4/toy_rlct_estimated.csv",
                    rlct_estimates_sgnht=rlct_estimates_sgnht,
                    rlct_estimates_sgld=rlct_estimates_sgld)
plot_lambda_vs_epochs(train_losses_toy_xp1_14, val_losses_toy_xp1_14, rlct_estimates_sgnht, rlct_estimates_sgld)


### Settings and runs label sets $Card(\mathcal{L}abels)=5$

* Labels modification and data loader
* Toy model for 5 classes
* Resnet18 model for 5 classes

In [ ]:
# keys are the original label, values are the new labels
# {0, 1, 4, 5, other} ; {0, 1, 7, 8, other} ; {4, 5, 7, 9, other} ; {2, 3, 6, 7, other}
map_label_card_5_1 = {0: 1, 1: 2, 2: 0, 3: 0, 4: 3, 5: 4, 6: 0, 7: 0, 8: 0, 9: 0}
map_label_card_5_2 = {0: 1, 1: 2, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 3, 8: 4, 9: 0}
map_label_card_5_3 = {0: 0, 1: 0, 2: 0, 3: 0, 4: 1, 5: 2, 6: 0, 7: 3, 8: 0, 9: 4}
map_label_card_5_4 = {0: 0, 1: 0, 2: 1, 3: 2, 4: 0, 5: 0, 6: 3, 7: 4, 8: 0, 9: 0}

##### 4.1 XP1 - labels set $\mathcal{Card} = 5$ - $\mathcal{L}abels=\{0, 1, 4, 5, other\}$

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# ________________________    Data Loader    ________________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
## Load train data
batch_size = 512
train_loader, test_loader = load_and_process_data(bs=batch_size,
                                                  mapping_label_to_encode=map_label_card_5_1)


# -------------------------------------------------------------------
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
n_classes = 5
n_epochs = 20
lr = 0.005
dict_optim_cfg = {"momentum":0.9 ,"nesterov": True}
# -------------------------------------------------------------------
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
if run_toy_model:
    model_toy, optimizer_toy, criterion_toy = models_factory(model_type="toy_model",
                                                            optim_cfg=dict_optim_cfg,
                                                            nb_classes=n_classes)

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 instance    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# Initialize model, loss, optimizer and sgld sampler
## taking an untrained resnet18
if run_resnet:
    models_resnet, optimizer_resnet, criterion_resnet = models_factory(model_type="toy_model",
                                                                    optim_cfg=dict_optim_cfg,
                                                                    nb_classes=n_classes)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 training    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
if run_resnet:
    train_losses_resnet_xp1_15 = []
    val_losses_resnet_xp1_15 = []
    models_resnet_xp1_15 = []
    for epoch in range(n_epochs):
        train_loss, train_acc = train_step(model_resnet18,
                                        optimizer_resnet18,
                                        criterion_resnet18,
                                        train_loader)
        val_loss, val_acc = val_step(model_resnet18,
                                    criterion_resnet18,
                                    test_loader)
        train_losses_resnet_xp1_15.append(train_loss)
        val_losses_resnet_xp1_15.append(val_loss)
        models_resnet_xp1_15 += [copy.deepcopy(model_resnet18)]

        print_step_report(epoch=epoch,
                        train_loss=train_loss,
                        val_loss=val_loss,
                        train_acc=train_acc,
                        val_acc=val_acc)

    torch.save(models_resnet_xp1_15, "XP1/card_5-case_1/resnet_models.pth")

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model training    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
if run_toy_model:
    train_losses_toy_xp1_15 = []
    val_losses_toy_xp1_15 = []
    train_acc_toy_xp1_15 = []
    val_acc_toy_xp1_15 = []
    models_toy_xp1_15 = []
    for epoch in range(n_epochs):
        train_loss, train_acc = train_step(model_toy,
                                        optimizer_toy,
                                        criterion_toy,
                                        train_loader)
        val_loss, val_acc = val_step(model_toy,
                                    criterion_toy,
                                    test_loader)
        train_losses_toy_xp1_15.append(train_loss)
        val_losses_toy_xp1_15.append(val_loss)
        train_acc_toy_xp1_15.append(train_acc)
        val_acc_toy_xp1_15.append(val_acc)
        models_toy_xp1_15 += [copy.deepcopy(model_toy)]

        print_step_report(epoch=epoch,
                        train_loss=train_loss,
                        val_loss=val_loss,
                        train_acc=train_acc,
                        val_acc=val_acc)

    torch.save(models_toy_xp1_15, "XP1/card_5-case_1/toy_models.pth")
    export_loss_acc_to_csv(path="XP1/card_5-case_1/toy_loss_acc.csv",
                                train_loss=train_losses_toy_xp1_15,
                                test_loss=val_losses_toy_xp1_15,
                                train_acc=train_acc_toy_xp1_15,
                                test_acc=val_acc_toy_xp1_15)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy Model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

rlct_estimates_sgnht = []
rlct_estimates_sgld = []
for model in models_toy_xp1_15:
    rlct_estimate_sgnht = estimate_learning_coeff(
        model,
        train_loader,
        criterion=criterion_toy,
        optimizer_kwargs=dict(
            lr=1e-7,
            diffusion_factor=0.01,
            num_samples=len(train_data),
        ),
        sampling_method=SGNHT,
        num_chains=1,
        num_draws=400,
        num_burnin_steps=0,
        num_steps_bw_draws=1,
        device=DEVICE,
    )
    rlct_estimate_sgld = estimate_learning_coeff(
        model,
        train_loader,
        criterion=criterion_toy,
        optimizer_kwargs=dict(
            lr=1e-5,
            noise_level=1.0,
            elasticity=100.0,
            num_samples=len(train_data),
            temperature="adaptive",
        ),
        sampling_method=SGLD,
        num_chains=1,
        num_draws=400,
        num_burnin_steps=0,
        num_steps_bw_draws=1,
        device=DEVICE,
    )
    rlct_estimates_sgnht += [rlct_estimate_sgnht]
    rlct_estimates_sgld += [rlct_estimate_sgld]
    print(rlct_estimate_sgld, rlct_estimate_sgnht)

# plotting
export_rlct_to_csv("XP1/card_5-case_1/toy_rlct_estimated.csv",
                    rlct_estimates_sgnht=rlct_estimates_sgnht,
                    rlct_estimates_sgld=rlct_estimates_sgld)
plot_lambda_vs_epochs(train_losses_toy_xp1_15, val_losses_toy_xp1_15, rlct_estimates_sgnht, rlct_estimates_sgld)


##### 4.2 XP1 - labels set $\mathcal{Card} = 5$ - $\mathcal{L}abels=\{0, 1, 7, 8, other\}$

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# ________________________    Data Loader    ________________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
## Load train data
batch_size = 512
train_loader, test_loader = load_and_process_data(bs=batch_size,
                                                  mapping_label_to_encode=map_label_card_5_2)


# -------------------------------------------------------------------
# DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
n_classes = 5
n_epochs = 20
lr = 0.005
dict_optim_cfg = {"momentum":0.9 ,"nesterov": True}
# -------------------------------------------------------------------
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
if run_toy_model:
    model_toy, optimizer_toy, criterion_toy = models_factory(model_type="toy_model",
                                                            optim_cfg=dict_optim_cfg,
                                                            nb_classes=n_classes)

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 instance    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# Initialize model, loss, optimizer and sgld sampler
## taking an untrained resnet18
if run_resnet:
    models_resnet, optimizer_resnet, criterion_resnet = models_factory(model_type="toy_model",
                                                                    optim_cfg=dict_optim_cfg,
                                                                    nb_classes=n_classes)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 training    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
if run_resnet:
    train_losses_resnet_xp1_16 = []
    val_losses_resnet_xp1_16 = []
    models_resnet_xp1_16 = []
    for epoch in range(n_epochs):
        train_loss, train_acc = train_step(model_resnet18,
                                        optimizer_resnet18,
                                        criterion_resnet18,
                                        train_loader)
        val_loss, val_acc = val_step(model_resnet18,
                                    criterion_resnet18,
                                    test_loader)
        train_losses_resnet_xp1_16.append(train_loss)
        val_losses_resnet_xp1_16.append(val_loss)
        models_resnet_xp1_16 += [copy.deepcopy(model_resnet18)]

        print_step_report(epoch=epoch,
                        train_loss=train_loss,
                        val_loss=val_loss,
                        train_acc=train_acc,
                        val_acc=val_acc)

    torch.save(models_resnet_xp1_16, "XP1/card_5-case_2/resnet_models.pth")

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model training    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
if run_toy_model:
    train_losses_toy_xp1_16 = []
    val_losses_toy_xp1_16 = []
    train_acc_toy_xp1_16 = []
    val_acc_toy_xp1_16 = []
    models_toy_xp1_16 = []
    for epoch in range(n_epochs):
        train_loss, train_acc = train_step(model_toy,
                                        optimizer_toy,
                                        criterion_toy,
                                        train_loader)
        val_loss, val_acc = val_step(model_toy,
                                    criterion_toy,
                                    test_loader)
        train_losses_toy_xp1_16.append(train_loss)
        val_losses_toy_xp1_16.append(val_loss)
        train_acc_toy_xp1_16.append(train_acc)
        val_acc_toy_xp1_16.append(val_acc)
        models_toy_xp1_16 += [copy.deepcopy(model_toy)]

        print_step_report(epoch=epoch,
                        train_loss=train_loss,
                        val_loss=val_loss,
                        train_acc=train_acc,
                        val_acc=val_acc)

    torch.save(models_toy_xp1_16, "XP1/card_5-case_2/toy_models.pth")
    export_loss_acc_to_csv(path="XP1/card_5-case_2/toy_loss_acc.csv",
                                train_loss=train_losses_toy_xp1_16,
                                test_loss=val_losses_toy_xp1_16,
                                train_acc=train_acc_toy_xp1_16,
                                test_acc=val_acc_toy_xp1_16)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy Model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

rlct_estimates_sgnht = []
rlct_estimates_sgld = []
for model in models_toy_xp1_16:
    rlct_estimate_sgnht = estimate_learning_coeff(
        model,
        train_loader,
        criterion=criterion_toy,
        optimizer_kwargs=dict(
            lr=1e-7,
            diffusion_factor=0.01,
            num_samples=len(train_data),
        ),
        sampling_method=SGNHT,
        num_chains=1,
        num_draws=400,
        num_burnin_steps=0,
        num_steps_bw_draws=1,
        device=DEVICE,
    )
    rlct_estimate_sgld = estimate_learning_coeff(
        model,
        train_loader,
        criterion=criterion_toy,
        optimizer_kwargs=dict(
            lr=1e-5,
            noise_level=1.0,
            elasticity=100.0,
            num_samples=len(train_data),
            temperature="adaptive",
        ),
        sampling_method=SGLD,
        num_chains=1,
        num_draws=400,
        num_burnin_steps=0,
        num_steps_bw_draws=1,
        device=DEVICE,
    )
    rlct_estimates_sgnht += [rlct_estimate_sgnht]
    rlct_estimates_sgld += [rlct_estimate_sgld]
    print(rlct_estimate_sgld, rlct_estimate_sgnht)

# plotting
export_rlct_to_csv("XP1/card_5-case_2/toy_rlct_estimated.csv",
                    rlct_estimates_sgnht=rlct_estimates_sgnht,
                    rlct_estimates_sgld=rlct_estimates_sgld)
plot_lambda_vs_epochs(train_losses_toy_xp1_16, val_losses_toy_xp1_16, rlct_estimates_sgnht, rlct_estimates_sgld)


##### 4.3 XP1 - labels set $\mathcal{Card} = 5$ - $\mathcal{L}abels=\{4, 5, 7, 9, other\}$

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# ________________________    Data Loader    ________________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
## Load train data
batch_size = 512
train_loader, test_loader = load_and_process_data(bs=batch_size,
                                                  mapping_label_to_encode=map_label_card_5_3)


# -------------------------------------------------------------------
# DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
n_classes = 5
n_epochs = 20
lr = 0.005
dict_optim_cfg = {"momentum":0.9 ,"nesterov": True}
# -------------------------------------------------------------------
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
if run_toy_model:
    model_toy, optimizer_toy, criterion_toy = models_factory(model_type="toy_model",
                                                            optim_cfg=dict_optim_cfg,
                                                            nb_classes=n_classes)

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 instance    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# Initialize model, loss, optimizer and sgld sampler
## taking an untrained resnet18
if run_resnet:
    models_resnet, optimizer_resnet, criterion_resnet = models_factory(model_type="toy_model",
                                                                    optim_cfg=dict_optim_cfg,
                                                                    nb_classes=n_classes)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 training    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
if run_resnet:
    train_losses_resnet_xp1_17 = []
    val_losses_resnet_xp1_17 = []
    models_resnet_xp1_17 = []
    for epoch in range(n_epochs):
        train_loss, train_acc = train_step(model_resnet18,
                                        optimizer_resnet18,
                                        criterion_resnet18,
                                        train_loader)
        val_loss, val_acc = val_step(model_resnet18,
                                    criterion_resnet18,
                                    test_loader)
        train_losses_resnet_xp1_17.append(train_loss)
        val_losses_resnet_xp1_17.append(val_loss)
        models_resnet_xp1_17 += [copy.deepcopy(model_resnet18)]

        print_step_report(epoch=epoch,
                        train_loss=train_loss,
                        val_loss=val_loss,
                        train_acc=train_acc,
                        val_acc=val_acc)

    torch.save(models_resnet_xp1_17, "XP1/card_5-case_3/resnet_models.pth")

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model training    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
if run_toy_model:
    train_losses_toy_xp1_17 = []
    val_losses_toy_xp1_17 = []
    train_acc_toy_xp1_17 = []
    val_acc_toy_xp1_17 = []
    models_toy_xp1_17 = []
    for epoch in range(n_epochs):
        train_loss, train_acc = train_step(model_toy,
                                        optimizer_toy,
                                        criterion_toy,
                                        train_loader)
        val_loss, val_acc = val_step(model_toy,
                                    criterion_toy,
                                    test_loader)
        train_losses_toy_xp1_17.append(train_loss)
        val_losses_toy_xp1_17.append(val_loss)
        train_acc_toy_xp1_17.append(train_acc)
        val_acc_toy_xp1_17.append(val_acc)
        models_toy_xp1_17 += [copy.deepcopy(model_toy)]

        print_step_report(epoch=epoch,
                        train_loss=train_loss,
                        val_loss=val_loss,
                        train_acc=train_acc,
                        val_acc=val_acc)

    torch.save(models_toy_xp1_17, "XP1/card_5-case_3/toy_models.pth")
    export_loss_acc_to_csv(path="XP1/card_5-case_3/toy_loss_acc.csv",
                                train_loss=train_losses_toy_xp1_17,
                                test_loss=val_losses_toy_xp1_17,
                                train_acc=train_acc_toy_xp1_17,
                                test_acc=val_acc_toy_xp1_17)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy Model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

rlct_estimates_sgnht = []
rlct_estimates_sgld = []
for model in models_toy_xp1_17:
    rlct_estimate_sgnht = estimate_learning_coeff(
        model,
        train_loader,
        criterion=criterion_toy,
        optimizer_kwargs=dict(
            lr=1e-7,
            diffusion_factor=0.01,
            num_samples=len(train_data),
        ),
        sampling_method=SGNHT,
        num_chains=1,
        num_draws=400,
        num_burnin_steps=0,
        num_steps_bw_draws=1,
        device=DEVICE,
    )
    rlct_estimate_sgld = estimate_learning_coeff(
        model,
        train_loader,
        criterion=criterion_toy,
        optimizer_kwargs=dict(
            lr=1e-5,
            noise_level=1.0,
            elasticity=100.0,
            num_samples=len(train_data),
            temperature="adaptive",
        ),
        sampling_method=SGLD,
        num_chains=1,
        num_draws=400,
        num_burnin_steps=0,
        num_steps_bw_draws=1,
        device=DEVICE,
    )
    rlct_estimates_sgnht += [rlct_estimate_sgnht]
    rlct_estimates_sgld += [rlct_estimate_sgld]
    print(rlct_estimate_sgld, rlct_estimate_sgnht)

# plotting
export_rlct_to_csv("XP1/card_5-case_3/toy_rlct_estimated.csv",
                    rlct_estimates_sgnht=rlct_estimates_sgnht,
                    rlct_estimates_sgld=rlct_estimates_sgld)
plot_lambda_vs_epochs(train_losses_toy_xp1_17, val_losses_toy_xp1_17, rlct_estimates_sgnht, rlct_estimates_sgld)


##### 4.4 XP1 - labels set $\mathcal{Card} = 5$ - $\mathcal{L}abels=\{2, 3, 6, 7, other\}$

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# ________________________    Data Loader    ________________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
## Load train data
batch_size = 512
train_loader, test_loader = load_and_process_data(bs=batch_size,
                                                  mapping_label_to_encode=map_label_card_5_4)


# -------------------------------------------------------------------
# DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
n_classes = 5
n_epochs = 20
lr = 0.005
dict_optim_cfg = {"momentum":0.9 ,"nesterov": True}
# -------------------------------------------------------------------
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
if run_toy_model:
    model_toy, optimizer_toy, criterion_toy = models_factory(model_type="toy_model",
                                                            optim_cfg=dict_optim_cfg,
                                                            nb_classes=n_classes)

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 instance    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# Initialize model, loss, optimizer and sgld sampler
## taking an untrained resnet18
if run_resnet:
    models_resnet, optimizer_resnet, criterion_resnet = models_factory(model_type="toy_model",
                                                                    optim_cfg=dict_optim_cfg,
                                                                    nb_classes=n_classes)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 training    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
if run_resnet:
    train_losses_resnet_xp1_18 = []
    val_losses_resnet_xp1_18 = []
    models_resnet_xp1_18 = []
    for epoch in range(n_epochs):
        train_loss, train_acc = train_step(model_resnet18,
                                        optimizer_resnet18,
                                        criterion_resnet18,
                                        train_loader)
        val_loss, val_acc = val_step(model_resnet18,
                                    criterion_resnet18,
                                    test_loader)
        train_losses_resnet_xp1_18.append(train_loss)
        val_losses_resnet_xp1_18.append(val_loss)
        models_resnet_xp1_18 += [copy.deepcopy(model_resnet18)]

        print_step_report(epoch=epoch,
                        train_loss=train_loss,
                        val_loss=val_loss,
                        train_acc=train_acc,
                        val_acc=val_acc)

    torch.save(models_resnet_xp1_18, "XP1/card_5-case_4/resnet_models.pth")

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model training    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
if run_toy_model:
    train_losses_toy_xp1_18 = []
    val_losses_toy_xp1_18 = []
    train_acc_toy_xp1_18 = []
    val_acc_toy_xp1_18 = []
    models_toy_xp1_18 = []
    for epoch in range(n_epochs):
        train_loss, train_acc = train_step(model_toy,
                                        optimizer_toy,
                                        criterion_toy,
                                        train_loader)
        val_loss, val_acc = val_step(model_toy,
                                    criterion_toy,
                                    test_loader)
        train_losses_toy_xp1_18.append(train_loss)
        val_losses_toy_xp1_18.append(val_loss)
        train_acc_toy_xp1_18.append(train_acc)
        val_acc_toy_xp1_18.append(val_acc)
        models_toy_xp1_18 += [copy.deepcopy(model_toy)]

        print_step_report(epoch=epoch,
                        train_loss=train_loss,
                        val_loss=val_loss,
                        train_acc=train_acc,
                        val_acc=val_acc)

    torch.save(models_toy_xp1_18, "XP1/card_5-case_4/toy_models.pth")
    export_loss_acc_to_csv(path="XP1/card_5-case_4/toy_loss_acc.csv",
                                train_loss=train_losses_toy_xp1_18,
                                test_loss=val_losses_toy_xp1_18,
                                train_acc=train_acc_toy_xp1_18,
                                test_acc=val_acc_toy_xp1_18)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy Model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

rlct_estimates_sgnht = []
rlct_estimates_sgld = []
for model in models_toy_xp1_18:
    rlct_estimate_sgnht = estimate_learning_coeff(
        model,
        train_loader,
        criterion=criterion_toy,
        optimizer_kwargs=dict(
            lr=1e-7,
            diffusion_factor=0.01,
            num_samples=len(train_data),
        ),
        sampling_method=SGNHT,
        num_chains=1,
        num_draws=400,
        num_burnin_steps=0,
        num_steps_bw_draws=1,
        device=DEVICE,
    )
    rlct_estimate_sgld = estimate_learning_coeff(
        model,
        train_loader,
        criterion=criterion_toy,
        optimizer_kwargs=dict(
            lr=1e-5,
            noise_level=1.0,
            elasticity=100.0,
            num_samples=len(train_data),
            temperature="adaptive",
        ),
        sampling_method=SGLD,
        num_chains=1,
        num_draws=400,
        num_burnin_steps=0,
        num_steps_bw_draws=1,
        device=DEVICE,
    )
    rlct_estimates_sgnht += [rlct_estimate_sgnht]
    rlct_estimates_sgld += [rlct_estimate_sgld]
    print(rlct_estimate_sgld, rlct_estimate_sgnht)

# plotting
export_rlct_to_csv("XP1/card_5-case_4/toy_rlct_estimated.csv",
                    rlct_estimates_sgnht=rlct_estimates_sgnht,
                    rlct_estimates_sgld=rlct_estimates_sgld)
plot_lambda_vs_epochs(train_losses_toy_xp1_18, val_losses_toy_xp1_18, rlct_estimates_sgnht, rlct_estimates_sgld)


### Settings and runs label sets 𝐶𝑎𝑟𝑑($\mathcal{L}$𝑎𝑏𝑒𝑙𝑠)=6

* Labels modification and data loader
* Toy model for 6 classes
* Resnet18 model for 6 classes


In [ ]:
# keys are the original label, values are the new labels
# {1, 2, 3, 6, 9, other} ; {3, 4, 7, 8, 9, other} ; {1, 3, 5, 7, 8, other} ; {0, 2, 4, 6, 7, other}
map_label_card_6_1 = {0: 0, 1: 1, 2: 2, 3: 3, 4: 0, 5: 0, 6: 4, 7: 0, 8: 0, 9: 5}
map_label_card_6_2 = {0: 0, 1: 0, 2: 0, 3: 1, 4: 2, 5: 0, 6: 0, 7: 3, 8: 4, 9: 5}
map_label_card_6_3 = {0: 0, 1: 1, 2: 0, 3: 2, 4: 0, 5: 3, 6: 0, 7: 4, 8: 5, 9: 0}
map_label_card_6_4 = {0: 1, 1: 0, 2: 2, 3: 0, 4: 3, 5: 0, 6: 4, 7: 5, 8: 0, 9: 0}

##### 5.1 XP1 - labels set $\mathcal{Card} = 6$ - $\mathcal{L}abels=\{1, 2, 3, 6, 9, other\}$

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# ________________________    Data Loader    ________________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
## Load train data
batch_size = 512
train_loader, test_loader = load_and_process_data(bs=batch_size,
                                                  mapping_label_to_encode=map_label_card_6_1)


# -------------------------------------------------------------------
# DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
n_classes = 6
n_epochs = 20
lr = 0.005
dict_optim_cfg = {"momentum":0.9 ,"nesterov": True}
# -------------------------------------------------------------------
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
if run_toy_model:
    model_toy, optimizer_toy, criterion_toy = models_factory(model_type="toy_model",
                                                            optim_cfg=dict_optim_cfg,
                                                            nb_classes=n_classes)

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 instance    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# Initialize model, loss, optimizer and sgld sampler
## taking an untrained resnet18
if run_resnet:
    models_resnet, optimizer_resnet, criterion_resnet = models_factory(model_type="toy_model",
                                                                    optim_cfg=dict_optim_cfg,
                                                                    nb_classes=n_classes)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 training    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
if run_resnet:
    train_losses_resnet_xp1_19 = []
    val_losses_resnet_xp1_19 = []
    models_resnet_xp1_19 = []
    for epoch in range(n_epochs):
        train_loss, train_acc = train_step(model_resnet18,
                                        optimizer_resnet18,
                                        criterion_resnet18,
                                        train_loader)
        val_loss, val_acc = val_step(model_resnet18,
                                    criterion_resnet18,
                                    test_loader)
        train_losses_resnet_xp1_19.append(train_loss)
        val_losses_resnet_xp1_19.append(val_loss)
        models_resnet_xp1_19 += [copy.deepcopy(model_resnet18)]

        print_step_report(epoch=epoch,
                        train_loss=train_loss,
                        val_loss=val_loss,
                        train_acc=train_acc,
                        val_acc=val_acc)

    torch.save(models_resnet_xp1_19, "XP1/card_6-case_1/resnet_models.pth")

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model training    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
if run_toy_model:
    train_losses_toy_xp1_19 = []
    val_losses_toy_xp1_19 = []
    train_acc_toy_xp1_19 = []
    val_acc_toy_xp1_19 = []
    models_toy_xp1_19 = []
    for epoch in range(n_epochs):
        train_loss, train_acc = train_step(model_toy,
                                        optimizer_toy,
                                        criterion_toy,
                                        train_loader)
        val_loss, val_acc = val_step(model_toy,
                                    criterion_toy,
                                    test_loader)
        train_losses_toy_xp1_19.append(train_loss)
        val_losses_toy_xp1_19.append(val_loss)
        train_acc_toy_xp1_19.append(train_acc)
        val_acc_toy_xp1_19.append(val_acc)
        models_toy_xp1_19 += [copy.deepcopy(model_toy)]

        print_step_report(epoch=epoch,
                        train_loss=train_loss,
                        val_loss=val_loss,
                        train_acc=train_acc,
                        val_acc=val_acc)

    torch.save(models_toy_xp1_19, "XP1/card_6-case_1/toy_models.pth")
    export_loss_acc_to_csv(path="XP1/card_6-case_1/toy_loss_acc.csv",
                                train_loss=train_losses_toy_xp1_19,
                                test_loss=val_losses_toy_xp1_19,
                                train_acc=train_acc_toy_xp1_19,
                                test_acc=val_acc_toy_xp1_19)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy Model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

rlct_estimates_sgnht = []
rlct_estimates_sgld = []
for model in models_toy_xp1_19:
    rlct_estimate_sgnht = estimate_learning_coeff(
        model,
        train_loader,
        criterion=criterion_toy,
        optimizer_kwargs=dict(
            lr=1e-7,
            diffusion_factor=0.01,
            num_samples=len(train_data),
        ),
        sampling_method=SGNHT,
        num_chains=1,
        num_draws=400,
        num_burnin_steps=0,
        num_steps_bw_draws=1,
        device=DEVICE,
    )
    rlct_estimate_sgld = estimate_learning_coeff(
        model,
        train_loader,
        criterion=criterion_toy,
        optimizer_kwargs=dict(
            lr=1e-5,
            noise_level=1.0,
            elasticity=100.0,
            num_samples=len(train_data),
            temperature="adaptive",
        ),
        sampling_method=SGLD,
        num_chains=1,
        num_draws=400,
        num_burnin_steps=0,
        num_steps_bw_draws=1,
        device=DEVICE,
    )
    rlct_estimates_sgnht += [rlct_estimate_sgnht]
    rlct_estimates_sgld += [rlct_estimate_sgld]
    print(rlct_estimate_sgld, rlct_estimate_sgnht)

# plotting
export_rlct_to_csv("XP1/card_6-case_1/toy_rlct_estimated.csv",
                    rlct_estimates_sgnht=rlct_estimates_sgnht,
                    rlct_estimates_sgld=rlct_estimates_sgld)
plot_lambda_vs_epochs(train_losses_toy_xp1_19, val_losses_toy_xp1_19, rlct_estimates_sgnht, rlct_estimates_sgld)


##### 5.2 XP1 - labels set $\mathcal{Card} = 6$ - $\mathcal{L}abels=\{3, 4, 7, 8, 9, other\}$

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# ________________________    Data Loader    ________________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
## Load train data
batch_size = 512
train_loader, test_loader = load_and_process_data(bs=batch_size,
                                                  mapping_label_to_encode=map_label_card_6_2)


# -------------------------------------------------------------------
# DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
n_classes = 6
n_epochs = 20
lr = 0.005
dict_optim_cfg = {"momentum":0.9 ,"nesterov": True}
# -------------------------------------------------------------------
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
if run_toy_model:
    model_toy, optimizer_toy, criterion_toy = models_factory(model_type="toy_model",
                                                            optim_cfg=dict_optim_cfg,
                                                            nb_classes=n_classes)

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 instance    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# Initialize model, loss, optimizer and sgld sampler
## taking an untrained resnet18
if run_resnet:
    models_resnet, optimizer_resnet, criterion_resnet = models_factory(model_type="toy_model",
                                                                    optim_cfg=dict_optim_cfg,
                                                                    nb_classes=n_classes)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 training    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
if run_resnet:
    train_losses_resnet_xp1_20 = []
    val_losses_resnet_xp1_20 = []
    models_resnet_xp1_20 = []
    for epoch in range(n_epochs):
        train_loss, train_acc = train_step(model_resnet18,
                                        optimizer_resnet18,
                                        criterion_resnet18,
                                        train_loader)
        val_loss, val_acc = val_step(model_resnet18,
                                    criterion_resnet18,
                                    test_loader)
        train_losses_resnet_xp1_20.append(train_loss)
        val_losses_resnet_xp1_20.append(val_loss)
        models_resnet_xp1_20 += [copy.deepcopy(model_resnet18)]

        print_step_report(epoch=epoch,
                        train_loss=train_loss,
                        val_loss=val_loss,
                        train_acc=train_acc,
                        val_acc=val_acc)

    torch.save(models_resnet_xp1_20, "XP1/card_6-case_2/resnet_models.pth")

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model training    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
if run_toy_model:
    train_losses_toy_xp1_20 = []
    val_losses_toy_xp1_20 = []
    train_acc_toy_xp1_20 = []
    val_acc_toy_xp1_20 = []
    models_toy_xp1_20 = []
    for epoch in range(n_epochs):
        train_loss, train_acc = train_step(model_toy,
                                        optimizer_toy,
                                        criterion_toy,
                                        train_loader)
        val_loss, val_acc = val_step(model_toy,
                                    criterion_toy,
                                    test_loader)
        train_losses_toy_xp1_20.append(train_loss)
        val_losses_toy_xp1_20.append(val_loss)
        train_acc_toy_xp1_20.append(train_acc)
        val_acc_toy_xp1_20.append(val_acc)
        models_toy_xp1_20 += [copy.deepcopy(model_toy)]

        print_step_report(epoch=epoch,
                        train_loss=train_loss,
                        val_loss=val_loss,
                        train_acc=train_acc,
                        val_acc=val_acc)

    torch.save(models_toy_xp1_20, "XP1/card_6-case_2/toy_models.pth")
    export_loss_acc_to_csv(path="XP1/card_6-case_2/toy_loss_acc.csv",
                                train_loss=train_losses_toy_xp1_20,
                                test_loss=val_losses_toy_xp1_20,
                                train_acc=train_acc_toy_xp1_20,
                                test_acc=val_acc_toy_xp1_20)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy Model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

rlct_estimates_sgnht = []
rlct_estimates_sgld = []
for model in models_toy_xp1_20:
    rlct_estimate_sgnht = estimate_learning_coeff(
        model,
        train_loader,
        criterion=criterion_toy,
        optimizer_kwargs=dict(
            lr=1e-7,
            diffusion_factor=0.01,
            num_samples=len(train_data),
        ),
        sampling_method=SGNHT,
        num_chains=1,
        num_draws=400,
        num_burnin_steps=0,
        num_steps_bw_draws=1,
        device=DEVICE,
    )
    rlct_estimate_sgld = estimate_learning_coeff(
        model,
        train_loader,
        criterion=criterion_toy,
        optimizer_kwargs=dict(
            lr=1e-5,
            noise_level=1.0,
            elasticity=100.0,
            num_samples=len(train_data),
            temperature="adaptive",
        ),
        sampling_method=SGLD,
        num_chains=1,
        num_draws=400,
        num_burnin_steps=0,
        num_steps_bw_draws=1,
        device=DEVICE,
    )
    rlct_estimates_sgnht += [rlct_estimate_sgnht]
    rlct_estimates_sgld += [rlct_estimate_sgld]
    print(rlct_estimate_sgld, rlct_estimate_sgnht)

# plotting
export_rlct_to_csv("XP1/card_6-case_2/toy_rlct_estimated.csv",
                    rlct_estimates_sgnht=rlct_estimates_sgnht,
                    rlct_estimates_sgld=rlct_estimates_sgld)
plot_lambda_vs_epochs(train_losses_toy_xp1_20, val_losses_toy_xp1_20, rlct_estimates_sgnht, rlct_estimates_sgld)


##### 5.3 XP1 - labels set $\mathcal{Card} = 6$ - $\mathcal{L}abels=\{1, 3, 5, 7, 8, other\}$

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# ________________________    Data Loader    ________________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
## Load train data
batch_size = 512
train_loader, test_loader = load_and_process_data(bs=batch_size,
                                                  mapping_label_to_encode=map_label_card_6_3)


# -------------------------------------------------------------------
# DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
n_classes = 6
n_epochs = 20
lr = 0.005
dict_optim_cfg = {"momentum":0.9 ,"nesterov": True}
# -------------------------------------------------------------------
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
if run_toy_model:
    model_toy, optimizer_toy, criterion_toy = models_factory(model_type="toy_model",
                                                            optim_cfg=dict_optim_cfg,
                                                            nb_classes=n_classes)

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 instance    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# Initialize model, loss, optimizer and sgld sampler
## taking an untrained resnet18
if run_resnet:
    models_resnet, optimizer_resnet, criterion_resnet = models_factory(model_type="toy_model",
                                                                    optim_cfg=dict_optim_cfg,
                                                                    nb_classes=n_classes)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 training    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
if run_resnet:
    train_losses_resnet_xp1_21 = []
    val_losses_resnet_xp1_21 = []
    models_resnet_xp1_21 = []
    for epoch in range(n_epochs):
        train_loss, train_acc = train_step(model_resnet18,
                                        optimizer_resnet18,
                                        criterion_resnet18,
                                        train_loader)
        val_loss, val_acc = val_step(model_resnet18,
                                    criterion_resnet18,
                                    test_loader)
        train_losses_resnet_xp1_21.append(train_loss)
        val_losses_resnet_xp1_21.append(val_loss)
        models_resnet_xp1_21 += [copy.deepcopy(model_resnet18)]

        print_step_report(epoch=epoch,
                        train_loss=train_loss,
                        val_loss=val_loss,
                        train_acc=train_acc,
                        val_acc=val_acc)

    torch.save(models_resnet_xp1_21, "XP1/card_6-case_3/resnet_models.pth")

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model training    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
if run_toy_model:
    train_losses_toy_xp1_21 = []
    val_losses_toy_xp1_21 = []
    train_acc_toy_xp1_21 = []
    val_acc_toy_xp1_21 = []
    models_toy_xp1_21 = []
    for epoch in range(n_epochs):
        train_loss, train_acc = train_step(model_toy,
                                        optimizer_toy,
                                        criterion_toy,
                                        train_loader)
        val_loss, val_acc = val_step(model_toy,
                                    criterion_toy,
                                    test_loader)
        train_losses_toy_xp1_21.append(train_loss)
        val_losses_toy_xp1_21.append(val_loss)
        train_acc_toy_xp1_21.append(train_acc)
        val_acc_toy_xp1_21.append(val_acc)
        models_toy_xp1_21 += [copy.deepcopy(model_toy)]

        print_step_report(epoch=epoch,
                        train_loss=train_loss,
                        val_loss=val_loss,
                        train_acc=train_acc,
                        val_acc=val_acc)

    torch.save(models_toy_xp1_21, "XP1/card_6-case_3/toy_models.pth")
    export_loss_acc_to_csv(path="XP1/card_6-case_3/toy_loss_acc.csv",
                                train_loss=train_losses_toy_xp1_21,
                                test_loss=val_losses_toy_xp1_21,
                                train_acc=train_acc_toy_xp1_21,
                                test_acc=val_acc_toy_xp1_21)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy Model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

rlct_estimates_sgnht = []
rlct_estimates_sgld = []
for model in models_toy_xp1_21:
    rlct_estimate_sgnht = estimate_learning_coeff(
        model,
        train_loader,
        criterion=criterion_toy,
        optimizer_kwargs=dict(
            lr=1e-7,
            diffusion_factor=0.01,
            num_samples=len(train_data),
        ),
        sampling_method=SGNHT,
        num_chains=1,
        num_draws=400,
        num_burnin_steps=0,
        num_steps_bw_draws=1,
        device=DEVICE,
    )
    rlct_estimate_sgld = estimate_learning_coeff(
        model,
        train_loader,
        criterion=criterion_toy,
        optimizer_kwargs=dict(
            lr=1e-5,
            noise_level=1.0,
            elasticity=100.0,
            num_samples=len(train_data),
            temperature="adaptive",
        ),
        sampling_method=SGLD,
        num_chains=1,
        num_draws=400,
        num_burnin_steps=0,
        num_steps_bw_draws=1,
        device=DEVICE,
    )
    rlct_estimates_sgnht += [rlct_estimate_sgnht]
    rlct_estimates_sgld += [rlct_estimate_sgld]
    print(rlct_estimate_sgld, rlct_estimate_sgnht)

# plotting
export_rlct_to_csv("XP1/card_6-case_3/toy_rlct_estimated.csv",
                    rlct_estimates_sgnht=rlct_estimates_sgnht,
                    rlct_estimates_sgld=rlct_estimates_sgld)
plot_lambda_vs_epochs(train_losses_toy_xp1_21, val_losses_toy_xp1_21, rlct_estimates_sgnht, rlct_estimates_sgld)


##### 5.4 XP1 - labels set $\mathcal{Card} = 6$ - $\mathcal{L}abels=\{0, 2, 4, 6, 7, other\}$

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# ________________________    Data Loader    ________________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
## Load train data
batch_size = 512
train_loader, test_loader = load_and_process_data(bs=batch_size,
                                                  mapping_label_to_encode=map_label_card_6_4)


# -------------------------------------------------------------------
# DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
n_classes = 6
n_epochs = 20
lr = 0.005
dict_optim_cfg = {"momentum":0.9 ,"nesterov": True}
# -------------------------------------------------------------------
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
if run_toy_model:
    model_toy, optimizer_toy, criterion_toy = models_factory(model_type="toy_model",
                                                            optim_cfg=dict_optim_cfg,
                                                            nb_classes=n_classes)

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 instance    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# Initialize model, loss, optimizer and sgld sampler
## taking an untrained resnet18
if run_resnet:
    models_resnet, optimizer_resnet, criterion_resnet = models_factory(model_type="toy_model",
                                                                    optim_cfg=dict_optim_cfg,
                                                                    nb_classes=n_classes)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 training    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
if run_resnet:
    train_losses_resnet_xp1_22 = []
    val_losses_resnet_xp1_22 = []
    models_resnet_xp1_22 = []
    for epoch in range(n_epochs):
        train_loss, train_acc = train_step(model_resnet18,
                                        optimizer_resnet18,
                                        criterion_resnet18,
                                        train_loader)
        val_loss, val_acc = val_step(model_resnet18,
                                    criterion_resnet18,
                                    test_loader)
        train_losses_resnet_xp1_22.append(train_loss)
        val_losses_resnet_xp1_22.append(val_loss)
        models_resnet_xp1_22 += [copy.deepcopy(model_resnet18)]

        print_step_report(epoch=epoch,
                        train_loss=train_loss,
                        val_loss=val_loss,
                        train_acc=train_acc,
                        val_acc=val_acc)

    torch.save(models_resnet_xp1_22, "XP1/card_6-case_4/resnet_models.pth")

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model training    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
if run_toy_model:
    train_losses_toy_xp1_22 = []
    val_losses_toy_xp1_22 = []
    train_acc_toy_xp1_22 = []
    val_acc_toy_xp1_22 = []
    models_toy_xp1_22 = []
    for epoch in range(n_epochs):
        train_loss, train_acc = train_step(model_toy,
                                        optimizer_toy,
                                        criterion_toy,
                                        train_loader)
        val_loss, val_acc = val_step(model_toy,
                                    criterion_toy,
                                    test_loader)
        train_losses_toy_xp1_22.append(train_loss)
        val_losses_toy_xp1_22.append(val_loss)
        train_acc_toy_xp1_22.append(train_acc)
        val_acc_toy_xp1_22.append(val_acc)
        models_toy_xp1_22 += [copy.deepcopy(model_toy)]

        print_step_report(epoch=epoch,
                        train_loss=train_loss,
                        val_loss=val_loss,
                        train_acc=train_acc,
                        val_acc=val_acc)

    torch.save(models_toy_xp1_22, "XP1/card_6-case_4/toy_models.pth")
    export_loss_acc_to_csv(path="XP1/card_6-case_4/toy_loss_acc.csv",
                                train_loss=train_losses_toy_xp1_22,
                                test_loss=val_losses_toy_xp1_22,
                                train_acc=train_acc_toy_xp1_22,
                                test_acc=val_acc_toy_xp1_22)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy Model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

rlct_estimates_sgnht = []
rlct_estimates_sgld = []
for model in models_toy_xp1_22:
    rlct_estimate_sgnht = estimate_learning_coeff(
        model,
        train_loader,
        criterion=criterion_toy,
        optimizer_kwargs=dict(
            lr=1e-7,
            diffusion_factor=0.01,
            num_samples=len(train_data),
        ),
        sampling_method=SGNHT,
        num_chains=1,
        num_draws=400,
        num_burnin_steps=0,
        num_steps_bw_draws=1,
        device=DEVICE,
    )
    rlct_estimate_sgld = estimate_learning_coeff(
        model,
        train_loader,
        criterion=criterion_toy,
        optimizer_kwargs=dict(
            lr=1e-5,
            noise_level=1.0,
            elasticity=100.0,
            num_samples=len(train_data),
            temperature="adaptive",
        ),
        sampling_method=SGLD,
        num_chains=1,
        num_draws=400,
        num_burnin_steps=0,
        num_steps_bw_draws=1,
        device=DEVICE,
    )
    rlct_estimates_sgnht += [rlct_estimate_sgnht]
    rlct_estimates_sgld += [rlct_estimate_sgld]
    print(rlct_estimate_sgld, rlct_estimate_sgnht)

# plotting
export_rlct_to_csv("XP1/card_6-case_4/toy_rlct_estimated.csv",
                    rlct_estimates_sgnht=rlct_estimates_sgnht,
                    rlct_estimates_sgld=rlct_estimates_sgld)
plot_lambda_vs_epochs(train_losses_toy_xp1_22, val_losses_toy_xp1_22, rlct_estimates_sgnht, rlct_estimates_sgld)


### Settings and runs label sets $Card(\mathcal{L}abels)=10$

* Labels modification and data loader
* Toy model for 10 classes
* Resnet18 model for 10 classes

In [ ]:
# Depending of the run type, cuda or cpu mode is defined
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# --------------------------------------------
N_CLASSES = 10 # for the 10 class digits of MNIST
lr = 0.005
n_epochs = 20
# --------------------------------------------
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 instance    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

# Initialize model, loss, optimizer and sgld sampler
## taking an untrained resnet18
if run_resnet:
    model_resnet18 = resnet18(weights=None).to(DEVICE)

    ## changing the last fully connected layer of N neurons, each corresponding to a class.
    new_fc_act = nn.Linear(model_resnet18.fc.in_features, N_CLASSES)
    model_resnet18.fc = new_fc_act

    ## Changing the first conv2d layer to match the input image shape
    model_resnet18.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

    # Loss and optimizer definition
    criterion_resnet18 = nn.CrossEntropyLoss()
    optimizer_resnet18 = optim.SGD(model_resnet18.parameters(), lr=lr, momentum=0.9, nesterov=True)

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy Model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

if run_toy_model:
    # Initialize model, loss, optimizer and sgld sampler
    model_toy = ToyNet().to(DEVICE)
    criterion_toy = nn.CrossEntropyLoss()
    optimizer_toy = optim.SGD(model_toy.parameters(), lr=lr, momentum=0.9, nesterov=True)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy Model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

if run_toy_model:
    train_losses_toy_xp1_23 = []
    val_losses_toy_xp1_23 = []
    train_acc_toy_xp1_23 = []
    val_acc_toy_xp1_23 = []
    models_toy_xp1_23 = []
    for epoch in range(n_epochs):
        train_loss, train_acc = train_step(model_toy,
                                        optimizer_toy,
                                        criterion_toy,
                                        train_loader)
        val_loss, val_acc = val_step(model_toy,
                                    criterion_toy,
                                    test_loader)
        train_losses_toy_xp1_23.append(train_loss)
        val_losses_toy_xp1_23.append(val_loss)
        train_acc_toy_xp1_23.append(train_acc)
        val_acc_toy_xp1_23.append(val_acc)
        models_toy_xp1_23 += [copy.deepcopy(model_toy)]

        print_step_report(epoch=epoch,
                        train_loss=train_loss,
                        val_loss=val_loss,
                        train_acc=train_acc,
                        val_acc=val_acc)

torch.save(models_toy_xp1_23, "XP1/card_10-case_1/toy_models.pth")
export_loss_acc_to_csv(path="XP1/card_10-case_1/toy_loss_acc.csv",
                            train_loss=train_losses_toy_xp1_23,
                            test_loss=val_losses_toy_xp1_23,
                            train_acc=train_acc_toy_xp1_23,
                            test_acc=val_acc_toy_xp1_23)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy Model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

rlct_estimates_sgnht = []
rlct_estimates_sgld = []
for model in models_toy_xp1_23:
    rlct_estimate_sgnht = estimate_learning_coeff(
        model,
        train_loader,
        criterion=criterion_toy,
        optimizer_kwargs=dict(
            lr=1e-7,
            diffusion_factor=0.01,
            num_samples=len(train_data),
        ),
        sampling_method=SGNHT,
        num_chains=1,
        num_draws=400,
        num_burnin_steps=0,
        num_steps_bw_draws=1,
        device=DEVICE,
    )
    rlct_estimate_sgld = estimate_learning_coeff(
        model,
        train_loader,
        criterion=criterion_toy,
        optimizer_kwargs=dict(
            lr=1e-5,
            noise_level=1.0,
            elasticity=100.0,
            num_samples=len(train_data),
            temperature="adaptive",
        ),
        sampling_method=SGLD,
        num_chains=1,
        num_draws=400,
        num_burnin_steps=0,
        num_steps_bw_draws=1,
        device=DEVICE,
    )
    rlct_estimates_sgnht += [rlct_estimate_sgnht]
    rlct_estimates_sgld += [rlct_estimate_sgld]
    print(rlct_estimate_sgld, rlct_estimate_sgnht)

# plotting
export_rlct_to_csv("XP1/card_10-case_1/toy_rlct_estimated.csv",
                    rlct_estimates_sgnht=rlct_estimates_sgnht,
                    rlct_estimates_sgld=rlct_estimates_sgld)
plot_lambda_vs_epochs(train_losses_toy_xp1_23, val_losses_toy_xp1_23, rlct_estimates_sgnht, rlct_estimates_sgld)


### Settings and runs label sets $Card(\mathcal{L}abels)=8$

* Labels modification and data loader
* Toy model for 8 classes
* Resnet18 model for 8 classes

In [ ]:
# keys are the original label, values are the new labels
# {1, 2, 3, 6, 9, other} ; {3, 4, 7, 8, 9, other} ; {1, 3, 5, 7, 8, other} ; {0, 2, 4, 6, 7, other}
map_label_card_8_1 = {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 0, 9: 0}
map_label_card_8_2 = {0: 1, 1: 2, 2: 3, 3: 0, 4: 0, 5: 0, 6: 5, 7: 6, 8: 7, 9: 0}

# BEWARE !! Due to the method of relabelling their is a corner case not well handle leading to less than 8 labels for second mapping
# It has been excluded from the report

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# ________________________    Data Loader    ________________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
## Load train data
batch_size = 512
train_loader, test_loader = load_and_process_data(bs=batch_size,
                                                  mapping_label_to_encode=map_label_card_8_1)


# -------------------------------------------------------------------
# DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
n_classes = 8
n_epochs = 20
lr = 0.005
dict_optim_cfg = {"momentum":0.9 ,"nesterov": True}
# -------------------------------------------------------------------
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
if run_toy_model:
    model_toy, optimizer_toy, criterion_toy = models_factory(model_type="toy_model",
                                                            optim_cfg=dict_optim_cfg,
                                                            nb_classes=n_classes)

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 instance    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# Initialize model, loss, optimizer and sgld sampler
## taking an untrained resnet18
if run_resnet:
    models_resnet, optimizer_resnet, criterion_resnet = models_factory(model_type="toy_model",
                                                                    optim_cfg=dict_optim_cfg,
                                                                    nb_classes=n_classes)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy Model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

if run_toy_model:
    train_losses_toy_xp1_24 = []
    val_losses_toy_xp1_24 = []
    train_acc_toy_xp1_24 = []
    val_acc_toy_xp1_24 = []
    models_toy_xp1_24 = []
    for epoch in range(n_epochs):
        train_loss, train_acc = train_step(model_toy,
                                        optimizer_toy,
                                        criterion_toy,
                                        train_loader)
        val_loss, val_acc = val_step(model_toy,
                                    criterion_toy,
                                    test_loader)
        train_losses_toy_xp1_24.append(train_loss)
        val_losses_toy_xp1_24.append(val_loss)
        train_acc_toy_xp1_24.append(train_acc)
        val_acc_toy_xp1_24.append(val_acc)
        models_toy_xp1_24 += [copy.deepcopy(model_toy)]

        print_step_report(epoch=epoch,
                        train_loss=train_loss,
                        val_loss=val_loss,
                        train_acc=train_acc,
                        val_acc=val_acc)

torch.save(models_toy_xp1_24, "XP1/card_8-case_1/toy_models.pth")
export_loss_acc_to_csv(path="XP1/card_8-case_1/toy_loss_acc.csv",
                            train_loss=train_losses_toy_xp1_24,
                            test_loss=val_losses_toy_xp1_24,
                            train_acc=train_acc_toy_xp1_24,
                            test_acc=val_acc_toy_xp1_24)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy Model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

rlct_estimates_sgnht = []
rlct_estimates_sgld = []
for model in models_toy_xp1_24:
    rlct_estimate_sgnht = estimate_learning_coeff(
        model,
        train_loader,
        criterion=criterion_toy,
        optimizer_kwargs=dict(
            lr=1e-7,
            diffusion_factor=0.01,
            num_samples=len(train_data),
        ),
        sampling_method=SGNHT,
        num_chains=1,
        num_draws=400,
        num_burnin_steps=0,
        num_steps_bw_draws=1,
        device=DEVICE,
    )
    rlct_estimate_sgld = estimate_learning_coeff(
        model,
        train_loader,
        criterion=criterion_toy,
        optimizer_kwargs=dict(
            lr=1e-5,
            noise_level=1.0,
            elasticity=100.0,
            num_samples=len(train_data),
            temperature="adaptive",
        ),
        sampling_method=SGLD,
        num_chains=1,
        num_draws=400,
        num_burnin_steps=0,
        num_steps_bw_draws=1,
        device=DEVICE,
    )
    rlct_estimates_sgnht += [rlct_estimate_sgnht]
    rlct_estimates_sgld += [rlct_estimate_sgld]
    print(rlct_estimate_sgld, rlct_estimate_sgnht)

# plotting
export_rlct_to_csv("XP1/card_8-case_1/toy_rlct_estimated.csv",
                    rlct_estimates_sgnht=rlct_estimates_sgnht,
                    rlct_estimates_sgld=rlct_estimates_sgld)
plot_lambda_vs_epochs(train_losses_toy_xp1_24, val_losses_toy_xp1_24, rlct_estimates_sgnht, rlct_estimates_sgld)


In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# ________________________    Data Loader    ________________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
## Load train data
batch_size = 512
train_loader, test_loader = load_and_process_data(bs=batch_size,
                                                  mapping_label_to_encode=map_label_card_8_2)


# -------------------------------------------------------------------
# DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
n_classes = 8
n_epochs = 20
lr = 0.005
dict_optim_cfg = {"momentum":0.9 ,"nesterov": True}
# -------------------------------------------------------------------
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
if run_toy_model:
    model_toy, optimizer_toy, criterion_toy = models_factory(model_type="toy_model",
                                                            optim_cfg=dict_optim_cfg,
                                                            nb_classes=n_classes)

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 instance    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# Initialize model, loss, optimizer and sgld sampler
## taking an untrained resnet18
if run_resnet:
    models_resnet, optimizer_resnet, criterion_resnet = models_factory(model_type="toy_model",
                                                                    optim_cfg=dict_optim_cfg,
                                                                    nb_classes=n_classes)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy Model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

if run_toy_model:
    train_losses_toy_xp1_25 = []
    val_losses_toy_xp1_25 = []
    train_acc_toy_xp1_25 = []
    val_acc_toy_xp1_25 = []
    models_toy_xp1_25 = []
    for epoch in range(n_epochs):
        train_loss, train_acc = train_step(model_toy,
                                        optimizer_toy,
                                        criterion_toy,
                                        train_loader)
        val_loss, val_acc = val_step(model_toy,
                                    criterion_toy,
                                    test_loader)
        train_losses_toy_xp1_25.append(train_loss)
        val_losses_toy_xp1_25.append(val_loss)
        train_acc_toy_xp1_25.append(train_acc)
        val_acc_toy_xp1_25.append(val_acc)
        models_toy_xp1_25 += [copy.deepcopy(model_toy)]

        print_step_report(epoch=epoch,
                        train_loss=train_loss,
                        val_loss=val_loss,
                        train_acc=train_acc,
                        val_acc=val_acc)

torch.save(models_toy_xp1_25, "XP1/card_8-case_2/toy_models.pth")
export_loss_acc_to_csv(path="XP1/card_8-case_2/toy_loss_acc.csv",
                            train_loss=train_losses_toy_xp1_25,
                            test_loss=val_losses_toy_xp1_25,
                            train_acc=train_acc_toy_xp1_25,
                            test_acc=val_acc_toy_xp1_25)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy Model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

rlct_estimates_sgnht = []
rlct_estimates_sgld = []
for model in models_toy_xp1_25:
    rlct_estimate_sgnht = estimate_learning_coeff(
        model,
        train_loader,
        criterion=criterion_toy,
        optimizer_kwargs=dict(
            lr=1e-7,
            diffusion_factor=0.01,
            num_samples=len(train_data),
        ),
        sampling_method=SGNHT,
        num_chains=1,
        num_draws=400,
        num_burnin_steps=0,
        num_steps_bw_draws=1,
        device=DEVICE,
    )
    rlct_estimate_sgld = estimate_learning_coeff(
        model,
        train_loader,
        criterion=criterion_toy,
        optimizer_kwargs=dict(
            lr=1e-5,
            noise_level=1.0,
            elasticity=100.0,
            num_samples=len(train_data),
            temperature="adaptive",
        ),
        sampling_method=SGLD,
        num_chains=1,
        num_draws=400,
        num_burnin_steps=0,
        num_steps_bw_draws=1,
        device=DEVICE,
    )
    rlct_estimates_sgnht += [rlct_estimate_sgnht]
    rlct_estimates_sgld += [rlct_estimate_sgld]
    print(rlct_estimate_sgld, rlct_estimate_sgnht)

# plotting
export_rlct_to_csv("XP1/card_8-case_2/toy_rlct_estimated.csv",
                    rlct_estimates_sgnht=rlct_estimates_sgnht,
                    rlct_estimates_sgld=rlct_estimates_sgld)
plot_lambda_vs_epochs(train_losses_toy_xp1_25, val_losses_toy_xp1_25, rlct_estimates_sgnht, rlct_estimates_sgld)


### Settings and runs label sets $Card(\mathcal{L}abels)=9$

* Labels modification and data loader
* Toy model for 9 classes
* Resnet18 model for 9 classes

In [ ]:
# keys are the original label, values are the new labels
# {1, 2, 3, 6, 9, other} ; {3, 4, 7, 8, 9, other} ; {1, 3, 5, 7, 8, other} ; {0, 2, 4, 6, 7, other}
map_label_card_9_1 = {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 0}
map_label_card_9_2 = {0: 1, 1: 2, 2: 3, 3: 0, 4: 0, 5: 0, 6: 5, 7: 6, 8: 7, 9: 8}

# BEWARE !! Due to the method of relabelling their is a corner case not well handle leading to less than 9 labels for second mapping
# It has been excluded from the report

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# ________________________    Data Loader    ________________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
## Load train data
batch_size = 512
train_loader, test_loader = load_and_process_data(bs=batch_size,
                                                  mapping_label_to_encode=map_label_card_9_1)


# -------------------------------------------------------------------
# DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
n_classes = 9
n_epochs = 20
lr = 0.005
dict_optim_cfg = {"momentum":0.9 ,"nesterov": True}
# -------------------------------------------------------------------
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
if run_toy_model:
    model_toy, optimizer_toy, criterion_toy = models_factory(model_type="toy_model",
                                                            optim_cfg=dict_optim_cfg,
                                                            nb_classes=n_classes)

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 instance    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# Initialize model, loss, optimizer and sgld sampler
## taking an untrained resnet18
if run_resnet:
    models_resnet, optimizer_resnet, criterion_resnet = models_factory(model_type="toy_model",
                                                                    optim_cfg=dict_optim_cfg,
                                                                    nb_classes=n_classes)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy Model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

if run_toy_model:
    train_losses_toy_xp1_26 = []
    val_losses_toy_xp1_26 = []
    train_acc_toy_xp1_26 = []
    val_acc_toy_xp1_26 = []
    models_toy_xp1_26 = []
    for epoch in range(n_epochs):
        train_loss, train_acc = train_step(model_toy,
                                        optimizer_toy,
                                        criterion_toy,
                                        train_loader)
        val_loss, val_acc = val_step(model_toy,
                                    criterion_toy,
                                    test_loader)
        train_losses_toy_xp1_26.append(train_loss)
        val_losses_toy_xp1_26.append(val_loss)
        train_acc_toy_xp1_26.append(train_acc)
        val_acc_toy_xp1_26.append(val_acc)
        models_toy_xp1_26 += [copy.deepcopy(model_toy)]

        print_step_report(epoch=epoch,
                        train_loss=train_loss,
                        val_loss=val_loss,
                        train_acc=train_acc,
                        val_acc=val_acc)

torch.save(models_toy_xp1_26, "XP1/card_9-case_1/toy_models.pth")
export_loss_acc_to_csv(path="XP1/card_9-case_1/toy_loss_acc.csv",
                            train_loss=train_losses_toy_xp1_26,
                            test_loss=val_losses_toy_xp1_26,
                            train_acc=train_acc_toy_xp1_26,
                            test_acc=val_acc_toy_xp1_26)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy Model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

rlct_estimates_sgnht = []
rlct_estimates_sgld = []
for model in models_toy_xp1_26:
    rlct_estimate_sgnht = estimate_learning_coeff(
        model,
        train_loader,
        criterion=criterion_toy,
        optimizer_kwargs=dict(
            lr=1e-7,
            diffusion_factor=0.01,
            num_samples=len(train_data),
        ),
        sampling_method=SGNHT,
        num_chains=1,
        num_draws=400,
        num_burnin_steps=0,
        num_steps_bw_draws=1,
        device=DEVICE,
    )
    rlct_estimate_sgld = estimate_learning_coeff(
        model,
        train_loader,
        criterion=criterion_toy,
        optimizer_kwargs=dict(
            lr=1e-5,
            noise_level=1.0,
            elasticity=100.0,
            num_samples=len(train_data),
            temperature="adaptive",
        ),
        sampling_method=SGLD,
        num_chains=1,
        num_draws=400,
        num_burnin_steps=0,
        num_steps_bw_draws=1,
        device=DEVICE,
    )
    rlct_estimates_sgnht += [rlct_estimate_sgnht]
    rlct_estimates_sgld += [rlct_estimate_sgld]
    print(rlct_estimate_sgld, rlct_estimate_sgnht)

# plotting
export_rlct_to_csv("XP1/card_9-case_1/toy_rlct_estimated.csv",
                    rlct_estimates_sgnht=rlct_estimates_sgnht,
                    rlct_estimates_sgld=rlct_estimates_sgld)
plot_lambda_vs_epochs(train_losses_toy_xp1_26, val_losses_toy_xp1_26, rlct_estimates_sgnht, rlct_estimates_sgld)


In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# ________________________    Data Loader    ________________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
## Load train data
batch_size = 512
train_loader, test_loader = load_and_process_data(bs=batch_size,
                                                  mapping_label_to_encode=map_label_card_9_2)


# -------------------------------------------------------------------
# DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
n_classes = 9
n_epochs = 20
lr = 0.005
dict_optim_cfg = {"momentum":0.9 ,"nesterov": True}
# -------------------------------------------------------------------
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
if run_toy_model:
    model_toy, optimizer_toy, criterion_toy = models_factory(model_type="toy_model",
                                                            optim_cfg=dict_optim_cfg,
                                                            nb_classes=n_classes)

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Resnet18 instance    _____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# Initialize model, loss, optimizer and sgld sampler
## taking an untrained resnet18
if run_resnet:
    models_resnet, optimizer_resnet, criterion_resnet = models_factory(model_type="toy_model",
                                                                    optim_cfg=dict_optim_cfg,
                                                                    nb_classes=n_classes)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy Model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

if run_toy_model:
    train_losses_toy_xp1_27 = []
    val_losses_toy_xp1_27 = []
    train_acc_toy_xp1_27 = []
    val_acc_toy_xp1_27 = []
    models_toy_xp1_27 = []
    for epoch in range(n_epochs):
        train_loss, train_acc = train_step(model_toy,
                                        optimizer_toy,
                                        criterion_toy,
                                        train_loader)
        val_loss, val_acc = val_step(model_toy,
                                    criterion_toy,
                                    test_loader)
        train_losses_toy_xp1_27.append(train_loss)
        val_losses_toy_xp1_27.append(val_loss)
        train_acc_toy_xp1_27.append(train_acc)
        val_acc_toy_xp1_27.append(val_acc)
        models_toy_xp1_27 += [copy.deepcopy(model_toy)]

        print_step_report(epoch=epoch,
                        train_loss=train_loss,
                        val_loss=val_loss,
                        train_acc=train_acc,
                        val_acc=val_acc)

torch.save(models_toy_xp1_27, "XP1/card_9-case_2/toy_models.pth")
export_loss_acc_to_csv(path="XP1/card_9-case_2/toy_loss_acc.csv",
                            train_loss=train_losses_toy_xp1_27,
                            test_loss=val_losses_toy_xp1_27,
                            train_acc=train_acc_toy_xp1_27,
                            test_acc=val_acc_toy_xp1_27)

In [ ]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #
# _____________________    Toy Model instance    ____________________ #
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% #

rlct_estimates_sgnht = []
rlct_estimates_sgld = []
for model in models_toy_xp1_27:
    rlct_estimate_sgnht = estimate_learning_coeff(
        model,
        train_loader,
        criterion=criterion_toy,
        optimizer_kwargs=dict(
            lr=1e-7,
            diffusion_factor=0.01,
            num_samples=len(train_data),
        ),
        sampling_method=SGNHT,
        num_chains=1,
        num_draws=400,
        num_burnin_steps=0,
        num_steps_bw_draws=1,
        device=DEVICE,
    )
    rlct_estimate_sgld = estimate_learning_coeff(
        model,
        train_loader,
        criterion=criterion_toy,
        optimizer_kwargs=dict(
            lr=1e-5,
            noise_level=1.0,
            elasticity=100.0,
            num_samples=len(train_data),
            temperature="adaptive",
        ),
        sampling_method=SGLD,
        num_chains=1,
        num_draws=400,
        num_burnin_steps=0,
        num_steps_bw_draws=1,
        device=DEVICE,
    )
    rlct_estimates_sgnht += [rlct_estimate_sgnht]
    rlct_estimates_sgld += [rlct_estimate_sgld]
    print(rlct_estimate_sgld, rlct_estimate_sgnht)

# plotting
export_rlct_to_csv("XP1/card_9-case_2/toy_rlct_estimated.csv",
                    rlct_estimates_sgnht=rlct_estimates_sgnht,
                    rlct_estimates_sgld=rlct_estimates_sgld)
plot_lambda_vs_epochs(train_losses_toy_xp1_27, val_losses_toy_xp1_27, rlct_estimates_sgnht, rlct_estimates_sgld)


## Displays λ 𝑣𝑠 $\mathcal{L}$𝑎𝑏𝑒𝑙𝑠

In [ ]:
sns.set_style("whitegrid")

fig, ax1 = plt.subplots()
ax1.set_xlabel("Epoch")
ax1.set_ylabel("Loss", color=PRIMARY)
ax1.plot(train_losses, label="Train Loss", color=PRIMARY)
ax1.plot(test_losses, label="Test Loss", color=PRIMARY_LIGHT)
ax1.tick_params(axis="y", labelcolor=PRIMARY)
ax1.legend(loc="lower left")

ax2 = ax1.twinx()
ax2.set_ylabel(r"Local Learning Coefficient, $\hat \lambda$", color=SECONDARY)
ax2.plot(rlct_estimates_sgnht, label="SGNHT", color=SECONDARY)
ax2.plot(rlct_estimates_sgld, label="SGLD", color=SECONDARY_LIGHT)
ax2.tick_params(axis="y", labelcolor=SECONDARY)
ax2.legend(loc="center right")

fig.tight_layout()
plt.show()